In [102]:
import numpy as np

In [103]:
def get_error(x, y):
    error = x - y
    print( np.linalg.norm(error.flatten()) )

In [104]:
from pyscf import gto, scf

from ccpy.models.calculation import Calculation
from ccpy.interfaces.pyscf_tools import load_pyscf_integrals
from ccpy.drivers.driver import cc_driver

mol = gto.Mole()
mol.build(
    atom="""F 0.0 0.0 -2.66816
            F 0.0 0.0  2.66816""",
    basis="ccpvdz",
    charge=0,
    spin=0,
    symmetry="D2H",
    cart=True,
    unit='Bohr',
)
mf = scf.ROHF(mol)
mf.kernel()


system, H0 = load_pyscf_integrals(mf, nfrozen=2)
system.print_info()

converged SCF energy = -198.420096282673
   System Information:
   ----------------------------------------------------
     Number of correlated electrons = 14
     Number of correlated orbitals = 28
     Number of frozen orbitals = 2
     Number of alpha occupied orbitals = 7
     Number of alpha unoccupied orbitals = 21
     Number of beta occupied orbitals = 7
     Number of beta unoccupied orbitals = 21
     Charge = 0
     Point group = D2h
     Symmetry of reference = AG
     Spin multiplicity of reference = 1

      MO #        Energy (a.u.)      Symmetry    Occupation
-----------------------------------------------------------
         1           -26.443066            AG           2.0
         2           -26.443033           B1U           2.0
         3            -1.597219            AG           2.0
         4            -1.589112           B1U           2.0
         5            -0.725614           B2U           2.0
         6            -0.725614           B3U           

In [105]:
calculation = Calculation(
    order=3,
    active_orders=[None],
    num_active=[None],
    calculation_type="ccsdt",
    convergence_tolerance=1.0e-08
)

T0, total_energy, is_converged = cc_driver(calculation, system, H0)

   --------------------------------------------------
   Calculation type =  CCSDT
   Maximum iterations = 60
   Convergence tolerance = 1e-08
   Energy shift = 0.0
   DIIS size =  6
   RHF symmetry = False
   --------------------------------------------------

   CC calculation started at 2022-03-12 18:06:00 

   Energy of initial guess =         0.0000000000

                 Iter.         Residuum                 δE                  ΔE               Wall time
-----------------------------------------------------------------------------------------------------
                   0         0.7950811421        -0.5760307223        -0.5760307223          (0.0m 7.8s)
                   1         0.2600010113        -0.0470790932        -0.6231098155          (0.0m 9.1s)
                   2         0.0673753279        -0.0073211685        -0.6304309840          (0.0m 9.8s)
                   3         0.0268057665        -0.0056287055        -0.6360596895          (0.0m 9.0s)
           

In [106]:
noccupied_alpha = T0.a.shape[1]
nunoccupied_alpha = T0.a.shape[0]
noccupied_beta = T0.b.shape[1]
nunoccupied_beta = T0.b.shape[0]

nact_occupied_alpha = 4
nact_unoccupied_alpha = 10
nact_occupied_beta = 4
nact_unoccupied_beta = 10

nunact_occupied_alpha = noccupied_alpha - nact_occupied_alpha
nunact_unoccupied_alpha = nunoccupied_alpha - nact_unoccupied_alpha
nunact_occupied_beta = noccupied_beta - nact_occupied_beta
nunact_unoccupied_beta = nunoccupied_beta - nact_unoccupied_beta

oa = slice(0, noccupied_alpha - nact_occupied_alpha)
Oa = slice(noccupied_alpha - nact_occupied_alpha, noccupied_alpha)
va = slice(nact_unoccupied_alpha, nunoccupied_alpha)
Va = slice(0, nact_unoccupied_alpha)

ob = slice(0, noccupied_beta - nact_occupied_beta)
Ob = slice(noccupied_beta - nact_occupied_beta, noccupied_beta)
vb = slice(nact_unoccupied_beta, nunoccupied_beta)
Vb = slice(0, nact_unoccupied_beta)

shift = 0.0

In [107]:
# An additional step needs to be done. We need to zero all T3 amplitudes 
# outside of the active space before computing the full CCSDT updates.
# This is because the active-space CC updates assume that T3 is 0 outside of
# the active space. If this is not true in the full scheme, then the results
# will not match.

def active_hole(x, nocc, nact):
    if x < nocc - nact:
        return 0
    else:
        return 1
    
def active_particle(x, nact):
    if x < nact:
        return 1
    else:
        return 0
    
# assume active spaces for alpha and beta are the same for now
def is_active(a, b, c, i, j, k, nocc, nact_h, nact_p):
    h = 0
    p = 0
    for n in [i, j, k]:
        h += active_hole(n, nocc, nact_h)
    for n in [a, b, c]:
        p += active_particle(n, nact_p)
    if h >= 1 and p >=1:
        return True
    else:
        return False


for a in range(system.nunoccupied_alpha):
    for b in range(a+1, system.nunoccupied_alpha):
        for c in range(b+1, system.nunoccupied_alpha):
            for i in range(system.noccupied_alpha):
                for j in range(i+1, system.noccupied_alpha):
                    for k in range(j+1, system.noccupied_alpha):
                        if not is_active(a, b, c, i, j, k, 
                                    system.noccupied_alpha, 
                                    nact_occupied_alpha, nact_unoccupied_alpha):
                            T0.aaa[a, b, c, i, j, k] = 0.0
                            T0.aaa[a, b, c, j, i, k] = 0.0
                            T0.aaa[a, b, c, j, k, i] = 0.0
                            T0.aaa[a, b, c, i, k, j] = 0.0
                            T0.aaa[a, b, c, k, j, i] = 0.0
                            T0.aaa[a, b, c, k, i, j] = 0.0
                            
                            T0.aaa[a, c, b, i, j, k] = 0.0
                            T0.aaa[a, c, b, j, i, k] = 0.0
                            T0.aaa[a, c, b, j, k, i] = 0.0
                            T0.aaa[a, c, b, i, k, j] = 0.0
                            T0.aaa[a, c, b, k, j, i] = 0.0
                            T0.aaa[a, c, b, k, i, j] = 0.0
                            
                            T0.aaa[b, a, c, i, j, k] = 0.0
                            T0.aaa[b, a, c, j, i, k] = 0.0
                            T0.aaa[b, a, c, j, k, i] = 0.0
                            T0.aaa[b, a, c, i, k, j] = 0.0
                            T0.aaa[b, a, c, k, j, i] = 0.0
                            T0.aaa[b, a, c, k, i, j] = 0.0
                            
                            T0.aaa[b, c, a, i, j, k] = 0.0
                            T0.aaa[b, c, a, j, i, k] = 0.0
                            T0.aaa[b, c, a, j, k, i] = 0.0
                            T0.aaa[b, c, a, i, k, j] = 0.0
                            T0.aaa[b, c, a, k, j, i] = 0.0
                            T0.aaa[b, c, a, k, i, j] = 0.0
                            
                            T0.aaa[c, a, b, i, j, k] = 0.0
                            T0.aaa[c, a, b, j, i, k] = 0.0
                            T0.aaa[c, a, b, j, k, i] = 0.0
                            T0.aaa[c, a, b, i, k, j] = 0.0
                            T0.aaa[c, a, b, k, j, i] = 0.0
                            T0.aaa[c, a, b, k, i, j] = 0.0
                            
                            T0.aaa[c, b, a, i, j, k] = 0.0
                            T0.aaa[c, b, a, j, i, k] = 0.0
                            T0.aaa[c, b, a, j, k, i] = 0.0
                            T0.aaa[c, b, a, i, k, j] = 0.0
                            T0.aaa[c, b, a, k, j, i] = 0.0
                            T0.aaa[c, b, a, k, i, j] = 0.0
                            
                            
for a in range(system.nunoccupied_alpha):
    for b in range(a+1, system.nunoccupied_alpha):
        for c in range(system.nunoccupied_beta):
            for i in range(system.noccupied_alpha):
                for j in range(i+1, system.noccupied_alpha):
                    for k in range(system.noccupied_beta):
                        if not is_active(a, b, c, i, j, k, 
                                    system.noccupied_alpha, 
                                    nact_occupied_alpha, nact_unoccupied_alpha):
                            T0.aab[a, b, c, i, j, k] = 0.0
                            T0.aab[b, a, c, i, j, k] = 0.0
                            T0.aab[a, b, c, j, i, k] = 0.0
                            T0.aab[b, a, c, j, i, k] = 0.0

In [168]:
# Populating the active-space structure
from ccpy.models.operators import ClusterOperator

T = ClusterOperator(system, 3, active_orders=[3], num_active=[1])

# T.a
T.a = T0.a.copy()
# T.ab
T.b = T0.b.copy()
# T.aa
T.aa = T0.aa.copy()
# T.ab
T.ab = T0.ab.copy()
# T.bb
T.bb = T0.bb.copy()

# T.aaa
T.aaa.VVVOOO = T0.aaa[Va, Va, Va, Oa, Oa, Oa]
T.aaa.VVvOOO = T0.aaa[Va, Va, va, Oa, Oa, Oa]
T.aaa.VVVoOO = T0.aaa[Va, Va, Va, oa, Oa, Oa]
T.aaa.VVvoOO = T0.aaa[Va, Va, va, oa, Oa, Oa]
T.aaa.VvvOOO = T0.aaa[Va, va, va, Oa, Oa, Oa]
T.aaa.VVVooO = T0.aaa[Va, Va, Va, oa, oa, Oa]
T.aaa.VvvoOO = T0.aaa[Va, va, va, oa, Oa, Oa]
T.aaa.VVvooO = T0.aaa[Va, Va, va, oa, oa, Oa]
T.aaa.VvvooO = T0.aaa[Va, va, va, oa, oa, Oa]

# T.aab
T.aab.VVVOOO = T0.aab[Va,Va,Vb,Oa,Oa,Ob]
T.aab.VVVoOO = T0.aab[Va,Va,Vb,oa,Oa,Ob]
T.aab.VVVOOo = T0.aab[Va,Va,Vb,Oa,Oa,ob]
T.aab.VVVoOo = T0.aab[Va,Va,Vb,oa,Oa,ob]
T.aab.VVVooO = T0.aab[Va,Va,Vb,oa,oa,Ob]

T.aab.VvVOOO = T0.aab[Va,va,Vb,Oa,Oa,Ob]
T.aab.VvVoOO = T0.aab[Va,va,Vb,oa,Oa,Ob]
T.aab.VvVOOo = T0.aab[Va,va,Vb,Oa,Oa,ob]
T.aab.VvVoOo = T0.aab[Va,va,Vb,oa,Oa,ob]
T.aab.VvVooO = T0.aab[Va,va,Vb,oa,oa,Ob]

T.aab.VVvOOO = T0.aab[Va,Va,vb,Oa,Oa,Ob]
T.aab.VVvoOO = T0.aab[Va,Va,vb,oa,Oa,Ob]
T.aab.VVvOOo = T0.aab[Va,Va,vb,Oa,Oa,ob]
T.aab.VVvoOo = T0.aab[Va,Va,vb,oa,Oa,ob]
T.aab.VVvooO = T0.aab[Va,Va,vb,oa,oa,Ob]

T.aab.VvvOOO = T0.aab[Va,va,vb,Oa,Oa,Ob]
T.aab.VvvoOO = T0.aab[Va,va,vb,oa,Oa,Ob]
T.aab.VvvOOo = T0.aab[Va,va,vb,Oa,Oa,ob]
T.aab.VvvoOo = T0.aab[Va,va,vb,oa,Oa,ob]
T.aab.VvvooO = T0.aab[Va,va,vb,oa,oa,Ob]

T.aab.vvVOOO = T0.aab[va,va,Vb,Oa,Oa,Ob]
T.aab.vvVoOO = T0.aab[va,va,Vb,oa,Oa,Ob]
T.aab.vvVOOo = T0.aab[va,va,Vb,Oa,Oa,ob]
T.aab.vvVoOo = T0.aab[va,va,Vb,oa,Oa,ob]
T.aab.vvVooO = T0.aab[va,va,Vb,oa,oa,Ob]

# T.abb
T.abb.VVVOOO = T0.abb[Va, Vb, Vb, Oa, Ob, Ob]
T.abb.VVVoOO = T0.abb[Va, Vb, Vb, oa, Ob, Ob]
T.abb.VVVOoO = T0.abb[Va, Vb, Vb, Oa, ob, Ob]
T.abb.VVVooO = T0.abb[Va, Vb, Vb, oa, ob, Ob]
T.abb.VVVOoo = T0.abb[Va, Vb, Vb, Oa, ob, ob]

T.abb.vVVOOO = T0.abb[va, Vb, Vb, Oa, Ob, Ob]
T.abb.vVVoOO = T0.abb[va, Vb, Vb, oa, Ob, Ob]
T.abb.vVVOoO = T0.abb[va, Vb, Vb, Oa, ob, Ob]
T.abb.vVVooO = T0.abb[va, Vb, Vb, oa, ob, Ob]
T.abb.vVVOoo = T0.abb[va, Vb, Vb, Oa, ob, ob]

T.abb.VVvOOO = T0.abb[Va, Vb, vb, Oa, Ob, Ob]
T.abb.VVvoOO = T0.abb[Va, Vb, vb, oa, Ob, Ob]
T.abb.VVvOoO = T0.abb[Va, Vb, vb, Oa, ob, Ob]
T.abb.VVvooO = T0.abb[Va, Vb, vb, oa, ob, Ob]
T.abb.VVvOoo = T0.abb[Va, Vb, vb, Oa, ob, ob]

T.abb.vVvOOO = T0.abb[va, Vb, vb, Oa, Ob, Ob]
T.abb.vVvoOO = T0.abb[va, Vb, vb, oa, Ob, Ob]
T.abb.vVvOoO = T0.abb[va, Vb, vb, Oa, ob, Ob]
T.abb.vVvooO = T0.abb[va, Vb, vb, oa, ob, Ob]
T.abb.vVvOoo = T0.abb[va, Vb, vb, Oa, ob, ob]

T.abb.VvvOOO = T0.abb[Va, vb, vb, Oa, Ob, Ob]
T.abb.VvvoOO = T0.abb[Va, vb, vb, oa, Ob, Ob]
T.abb.VvvOoO = T0.abb[Va, vb, vb, Oa, ob, Ob]
T.abb.VvvooO = T0.abb[Va, vb, vb, oa, ob, Ob]
T.abb.VvvOoo = T0.abb[Va, vb, vb, Oa, ob, ob]

In [169]:
from ccpy.hbar.hbar_ccsd import build_hbar_ccsd
from ccpy.hbar.hbar_ccs import get_ccs_intermediates

H = build_hbar_ccsd(T0, H0)
H1 = get_ccs_intermediates(T0, H0)

In [170]:
I1A_oo = (
        H.a.oo
        + 0.5 * np.einsum("mnef,efin->mi", H1.aa.oovv, T.aa, optimize=True)
        + np.einsum("mnef,efin->mi", H1.ab.oovv, T.ab, optimize=True)
)

I1A_vv = (
        H.a.vv
        - 0.5 * np.einsum("mnef,afmn->ae", H1.aa.oovv, T.aa, optimize=True)
        - np.einsum("mnef,afmn->ae", H1.ab.oovv, T.ab, optimize=True)
)

I2A_voov = (
        H.aa.voov
        + 0.5 * np.einsum("mnef,afin->amie", H1.aa.oovv, T.aa, optimize=True)
        + np.einsum("mnef,afin->amie", H1.ab.oovv, T.ab, optimize=True)
)

I2A_oooo = H.aa.oooo + 0.5 * np.einsum(
    "mnef,efij->mnij", H1.aa.oovv, T.aa, optimize=True
)

I2B_voov = H.ab.voov + 0.5 * np.einsum(
    "mnef,afin->amie", H1.bb.oovv, T.ab, optimize=True
)

X2A = -0.5 * np.einsum("amij,bm->abij", H1.aa.vooo, T.a, optimize=True)
X2A += 0.5 * np.einsum("abie,ej->abij", H1.aa.vvov, T.a, optimize=True)
X2A += 0.5 * np.einsum("ae,ebij->abij", I1A_vv, T.aa, optimize=True)
X2A -= 0.5 * np.einsum("mi,abmj->abij", I1A_oo, T.aa, optimize=True)
X2A += np.einsum("amie,ebmj->abij", I2A_voov, T.aa, optimize=True)
X2A += np.einsum("amie,bejm->abij", I2B_voov, T.ab, optimize=True)
X2A += 0.125 * np.einsum("abef,efij->abij", H1.aa.vvvv, T.aa, optimize=True)
X2A += 0.125 * np.einsum("mnij,abmn->abij", I2A_oooo, T.aa, optimize=True)

X2A0 = X2A.copy()

X2A0 -= np.transpose(X2A0, (1, 0, 2, 3))
X2A0 -= np.transpose(X2A0, (0, 1, 3, 2))

In [171]:
X2A += 0.25 * np.einsum("me,abeijm->abij", H1.a.ov, T0.aaa, optimize=True)
X2A += 0.25 * np.einsum("me,abeijm->abij", H1.b.ov, T0.aab, optimize=True)
X2A -= 0.5 * np.einsum("mnif,abfmjn->abij", H1.ab.ooov, T0.aab, optimize=True)
X2A -= 0.25 * np.einsum("mnif,abfmjn->abij", H1.aa.ooov, T0.aaa, optimize=True)
X2A += 0.25 * np.einsum("anef,ebfijn->abij", H1.aa.vovv, T0.aaa, optimize=True)
X2A += 0.5 * np.einsum("anef,ebfijn->abij", H1.ab.vovv, T0.aab, optimize=True)

X2A -= np.transpose(X2A, (1, 0, 2, 3))
X2A -= np.transpose(X2A, (0, 1, 3, 2))

In [172]:
x2a = 0.0
x2a = (1.0 / 2.0) * (
        -1.0 * np.einsum('me,AebmIJ->AbIJ', H1.a.ov[oa, va], T.aaa.VvvoOO, optimize=True)
        + 1.0 * np.einsum('mE,EAbmIJ->AbIJ', H1.a.ov[oa, Va], T.aaa.VVvoOO, optimize=True)
        - 1.0 * np.einsum('Me,AebIJM->AbIJ', H1.a.ov[Oa, va], T.aaa.VvvOOO, optimize=True)
        + 1.0 * np.einsum('ME,EAbIJM->AbIJ', H1.a.ov[Oa, Va], T.aaa.VVvOOO, optimize=True)
)
x2a += (1.0 / 2.0) * (
        +1.0 * np.einsum('me,AbeIJm->AbIJ', H1.b.ov[ob, vb], T.aab.VvvOOo, optimize=True)
        + 1.0 * np.einsum('mE,AbEIJm->AbIJ', H1.b.ov[ob, Vb], T.aab.VvVOOo, optimize=True)
        + 1.0 * np.einsum('Me,AbeIJM->AbIJ', H1.b.ov[Ob, vb], T.aab.VvvOOO, optimize=True)
        + 1.0 * np.einsum('ME,AbEIJM->AbIJ', H1.b.ov[Ob, Vb], T.aab.VvVOOO, optimize=True)
)
x2a += (2.0 / 2.0) * (
        -1.0 * np.einsum('mnIf,AbfmJn->AbIJ', H1.ab.ooov[oa, ob, Oa, vb], T.aab.VvvoOo, optimize=True)
        - 1.0 * np.einsum('mnIF,AbFmJn->AbIJ', H1.ab.ooov[oa, ob, Oa, Vb], T.aab.VvVoOo, optimize=True)
        - 1.0 * np.einsum('mNIf,AbfmJN->AbIJ', H1.ab.ooov[oa, Ob, Oa, vb], T.aab.VvvoOO, optimize=True)
        - 1.0 * np.einsum('mNIF,AbFmJN->AbIJ', H1.ab.ooov[oa, Ob, Oa, Vb], T.aab.VvVoOO, optimize=True)
        - 1.0 * np.einsum('MnIf,AbfMJn->AbIJ', H1.ab.ooov[Oa, ob, Oa, vb], T.aab.VvvOOo, optimize=True)
        - 1.0 * np.einsum('MnIF,AbFMJn->AbIJ', H1.ab.ooov[Oa, ob, Oa, Vb], T.aab.VvVOOo, optimize=True)
        - 1.0 * np.einsum('MNIf,AbfMJN->AbIJ', H1.ab.ooov[Oa, Ob, Oa, vb], T.aab.VvvOOO, optimize=True)
        - 1.0 * np.einsum('MNIF,AbFMJN->AbIJ', H1.ab.ooov[Oa, Ob, Oa, Vb], T.aab.VvVOOO, optimize=True)
)
x2a += (2.0 / 2.0) * (
        -0.5 * np.einsum('mnIf,AfbmnJ->AbIJ', H1.aa.ooov[oa, oa, Oa, va], T.aaa.VvvooO, optimize=True)
        + 0.5 * np.einsum('mnIF,FAbmnJ->AbIJ', H1.aa.ooov[oa, oa, Oa, Va], T.aaa.VVvooO, optimize=True)
        + 1.0 * np.einsum('MnIf,AfbnMJ->AbIJ', H1.aa.ooov[Oa, oa, Oa, va], T.aaa.VvvoOO, optimize=True)
        - 1.0 * np.einsum('MnIF,FAbnMJ->AbIJ', H1.aa.ooov[Oa, oa, Oa, Va], T.aaa.VVvoOO, optimize=True)
        + 0.5 * np.einsum('MNIf,AfbMJN->AbIJ', H1.aa.ooov[Oa, Oa, Oa, va], T.aaa.VvvOOO, optimize=True)
        - 0.5 * np.einsum('MNIF,FAbMJN->AbIJ', H1.aa.ooov[Oa, Oa, Oa, Va], T.aaa.VVvOOO, optimize=True)
)
x2a += (1.0 / 2.0) * (
        -1.0 * np.einsum('AnEf,EfbnIJ->AbIJ', H1.aa.vovv[Va, oa, Va, va], T.aaa.VvvoOO, optimize=True)
        + 0.5 * np.einsum('AnEF,FEbnIJ->AbIJ', H1.aa.vovv[Va, oa, Va, Va], T.aaa.VVvoOO, optimize=True)
        - 1.0 * np.einsum('ANEf,EfbIJN->AbIJ', H1.aa.vovv[Va, Oa, Va, va], T.aaa.VvvOOO, optimize=True)
        + 0.5 * np.einsum('ANEF,FEbIJN->AbIJ', H1.aa.vovv[Va, Oa, Va, Va], T.aaa.VVvOOO, optimize=True)
)
x2a += (1.0 / 2.0) * (
        -0.5 * np.einsum('bnef,AfenIJ->AbIJ', H1.aa.vovv[va, oa, va, va], T.aaa.VvvoOO, optimize=True)
        - 0.5 * np.einsum('bNef,AfeIJN->AbIJ', H1.aa.vovv[va, Oa, va, va], T.aaa.VvvOOO, optimize=True)
        + 1.0 * np.einsum('bnEf,AEfnIJ->AbIJ', H1.aa.vovv[va, oa, Va, va], T.aaa.VVvoOO, optimize=True)
        + 0.5 * np.einsum('bnEF,FAEnIJ->AbIJ', H1.aa.vovv[va, oa, Va, Va], T.aaa.VVVoOO, optimize=True)
        + 1.0 * np.einsum('bNEf,AEfIJN->AbIJ', H1.aa.vovv[va, Oa, Va, va], T.aaa.VVvOOO, optimize=True)
        + 0.5 * np.einsum('bNEF,FAEIJN->AbIJ', H1.aa.vovv[va, Oa, Va, Va], T.aaa.VVVOOO, optimize=True)
)
x2a += (1.0 / 2.0) * (
        -1.0 * np.einsum('AneF,beFIJn->AbIJ', H1.ab.vovv[Va, ob, va, Vb], T.aab.vvVOOo, optimize=True)
        - 1.0 * np.einsum('ANeF,beFIJN->AbIJ', H1.ab.vovv[Va, Ob, va, Vb], T.aab.vvVOOO, optimize=True)
        + 1.0 * np.einsum('AnEf,EbfIJn->AbIJ', H1.ab.vovv[Va, ob, Va, vb], T.aab.VvvOOo, optimize=True)
        + 1.0 * np.einsum('AnEF,EbFIJn->AbIJ', H1.ab.vovv[Va, ob, Va, Vb], T.aab.VvVOOo, optimize=True)
        + 1.0 * np.einsum('ANEf,EbfIJN->AbIJ', H1.ab.vovv[Va, Ob, Va, vb], T.aab.VvvOOO, optimize=True)
        + 1.0 * np.einsum('ANEF,EbFIJN->AbIJ', H1.ab.vovv[Va, Ob, Va, Vb], T.aab.VvVOOO, optimize=True)
)
x2a += (1.0 / 2.0) * (
        +1.0 * np.einsum('bnef,AefIJn->AbIJ', H1.ab.vovv[va, ob, va, vb], T.aab.VvvOOo, optimize=True)
        + 1.0 * np.einsum('bneF,AeFIJn->AbIJ', H1.ab.vovv[va, ob, va, Vb], T.aab.VvVOOo, optimize=True)
        + 1.0 * np.einsum('bNef,AefIJN->AbIJ', H1.ab.vovv[va, Ob, va, vb], T.aab.VvvOOO, optimize=True)
        + 1.0 * np.einsum('bNeF,AeFIJN->AbIJ', H1.ab.vovv[va, Ob, va, Vb], T.aab.VvVOOO, optimize=True)
        + 1.0 * np.einsum('bnEf,AEfIJn->AbIJ', H1.ab.vovv[va, ob, Va, vb], T.aab.VVvOOo, optimize=True)
        + 1.0 * np.einsum('bnEF,AEFIJn->AbIJ', H1.ab.vovv[va, ob, Va, Vb], T.aab.VVVOOo, optimize=True)
        + 1.0 * np.einsum('bNEf,AEfIJN->AbIJ', H1.ab.vovv[va, Ob, Va, vb], T.aab.VVvOOO, optimize=True)
        + 1.0 * np.einsum('bNEF,AEFIJN->AbIJ', H1.ab.vovv[va, Ob, Va, Vb], T.aab.VVVOOO, optimize=True)
)

x2a -= np.transpose(x2a, (0, 1, 3, 2))

X2A0[Va, va, Oa, Oa] += x2a

In [173]:
get_error(X2A0[Va, va, Oa, Oa], X2A[Va, va, Oa, Oa])

2.787685417055999e-17


In [180]:
a = slice(0,5)
a

slice(0, 5, None)

In [9]:
from ccpy.utilities.updates import cc_loops2
from ccpy.utilities.updates import cc_active_loops

In [10]:
# The issue here is that the diagram with an extra weight factor is not actually equal to the
# underlying diagram with the correct antisymmetrizer applied. They are only equal once ALL
# antisymmetrizers are applied to the former diagram. That is the point of the weight factor! 
# As a result, you won't see equivalence until you apply the full antisymmetrizer within the 
# update loop.

D1 = -np.einsum("mk,abcijm->abcijk", H.a.oo, T0.aaa, optimize=True)
D1 -= np.transpose(D1, (0, 1, 2, 5, 4, 3)) + np.transpose(D1, (0, 1, 2, 3, 5, 4))

D2 = np.einsum('ae,ebcijk->abcijk', H.a.vv, T0.aaa, optimize=True)
D2 -= np.transpose(D2, (1, 0, 2, 3, 4, 5)) + np.transpose(D2, (2, 1, 0, 3, 4, 5))

D3 = 0.5 * np.einsum('mnij,abcmnk->abcijk', H.aa.oooo, T0.aaa, optimize=True)
D3 -= np.transpose(D3, (0, 1, 2, 5, 4, 3)) + np.transpose(D3, (0, 1, 2, 3, 5, 4))

D4 = 0.5 * np.einsum('abef,efcijk->abcijk', H.aa.vvvv, T0.aaa, optimize=True)
D4 -= np.transpose(D4, (2, 1, 0, 3, 4, 5)) + np.transpose(D4, (0, 2, 1, 3, 4, 5))

D5 = np.einsum('amie,ebcmjk->abcijk', H.aa.voov, T0.aaa, optimize=True)
D5 -= np.transpose(D5, (1, 0, 2, 3, 4, 5)) + np.transpose(D5, (2, 1, 0, 3, 4, 5))
D5 -= np.transpose(D5, (0, 1, 2, 4, 3, 5)) + np.transpose(D5, (0, 1, 2, 5, 4, 3))

D6 = np.einsum('amie,bcejkm->abcijk', H.ab.voov, T0.aab, optimize=True)
D6 -= np.transpose(D6, (1, 0, 2, 3, 4, 5)) + np.transpose(D6, (2, 1, 0, 3, 4, 5))
D6 -= np.transpose(D6, (0, 1, 2, 4, 3, 5)) + np.transpose(D6, (0, 1, 2, 5, 4, 3))


X3A = D1 + D2 + D3 + D4 + D5 + D6

## Projection 1

In [11]:
x3a = (3.0/36.0) * (
        +1.0*np.einsum('mI,CBAmJK->ABCIJK', H.a.oo[oa, Oa], T.aaa.VVVoOO, optimize=True)
        +1.0*np.einsum('MI,CBAMJK->ABCIJK', H.a.oo[Oa, Oa], T.aaa.VVVOOO, optimize=True)
)
x3a += (3.0/36.0) * (
        -1.0*np.einsum('Ae,CBeIJK->ABCIJK', H.a.vv[Va, va], T.aaa.VVvOOO, optimize=True)
        -1.0*np.einsum('AE,CBEIJK->ABCIJK', H.a.vv[Va, Va], T.aaa.VVVOOO, optimize=True)
)
x3a += (3.0/36.0) * (
        -0.5*np.einsum('mnIJ,CBAmnK->ABCIJK', H.aa.oooo[oa, oa, Oa, Oa], T.aaa.VVVooO, optimize=True)
        +1.0*np.einsum('MnIJ,CBAnMK->ABCIJK', H.aa.oooo[Oa, oa, Oa, Oa], T.aaa.VVVoOO, optimize=True)
        -0.5*np.einsum('MNIJ,CBAMNK->ABCIJK', H.aa.oooo[Oa, Oa, Oa, Oa], T.aaa.VVVOOO, optimize=True)
)
x3a += (3.0/36.0) * (
        -0.5*np.einsum('ABef,CfeIJK->ABCIJK', H.aa.vvvv[Va, Va, va, va], T.aaa.VvvOOO, optimize=True)
        -1.0*np.einsum('ABeF,CFeIJK->ABCIJK', H.aa.vvvv[Va, Va, va, Va], T.aaa.VVvOOO, optimize=True)
        -0.5*np.einsum('ABEF,CFEIJK->ABCIJK', H.aa.vvvv[Va, Va, Va, Va], T.aaa.VVVOOO, optimize=True)
)
x3a += (9.0/36.0) * (
        -1.0*np.einsum('AmIe,CBemJK->ABCIJK', H.aa.voov[Va, oa, Oa, va], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('AmIE,CBEmJK->ABCIJK', H.aa.voov[Va, oa, Oa, Va], T.aaa.VVVoOO, optimize=True)
        -1.0*np.einsum('AMIe,CBeMJK->ABCIJK', H.aa.voov[Va, Oa, Oa, va], T.aaa.VVvOOO, optimize=True)
        -1.0*np.einsum('AMIE,CBEMJK->ABCIJK', H.aa.voov[Va, Oa, Oa, Va], T.aaa.VVVOOO, optimize=True)
)
x3a += (9.0/36.0) * (
        -1.0*np.einsum('AmIe,CBeJKm->ABCIJK', H.ab.voov[Va, ob, Oa, vb], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('AmIE,CBEJKm->ABCIJK', H.ab.voov[Va, ob, Oa, Vb], T.aab.VVVOOo, optimize=True)
        -1.0*np.einsum('AMIe,CBeJKM->ABCIJK', H.ab.voov[Va, Ob, Oa, vb], T.aab.VVvOOO, optimize=True)
        -1.0*np.einsum('AMIE,CBEJKM->ABCIJK', H.ab.voov[Va, Ob, Oa, Vb], T.aab.VVVOOO, optimize=True)
)


x3a -= np.transpose(x3a, (1, 0, 2, 3, 4, 5)) + np.transpose(x3a, (0, 2, 1, 3, 4, 5))\
     + np.transpose(x3a, (2, 1, 0, 3, 4, 5)) - np.transpose(x3a, (1, 2, 0, 3, 4, 5))\
     - np.transpose(x3a, (2, 0, 1, 3, 4, 5))
        
x3a -= np.transpose(x3a, (0, 1, 2, 4, 3, 5)) + np.transpose(x3a, (0, 1, 2, 3, 5, 4))\
     + np.transpose(x3a, (0, 1, 2, 5, 4, 3)) - np.transpose(x3a, (0, 1, 2, 4, 5, 3))\
     - np.transpose(x3a, (0, 1, 2, 5, 3, 4))
        
get_error(X3A[Va, Va, Va, Oa, Oa, Oa], x3a)

1.1970590077769788e-17


### Projection 2

In [12]:
x3a = (3.0/12.0) * (
        +1.0*np.einsum('mI,BAcmJK->ABcIJK', H.a.oo[oa, Oa], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('MI,BAcMJK->ABcIJK', H.a.oo[Oa, Oa], T.aaa.VVvOOO, optimize=True)
)
x3a += (2.0/12.0) * (
        +1.0*np.einsum('Ae,BceIJK->ABcIJK', H.a.vv[Va, va], T.aaa.VvvOOO, optimize=True)
        -1.0*np.einsum('AE,BEcIJK->ABcIJK', H.a.vv[Va, Va], T.aaa.VVvOOO, optimize=True)
)
x3a += (1.0/12.0) * (
        +1.0*np.einsum('ce,ABeIJK->ABcIJK', H.a.vv[va, va], T.aaa.VVvOOO, optimize=True)
        +1.0*np.einsum('cE,ABEIJK->ABcIJK', H.a.vv[va, Va], T.aaa.VVVOOO, optimize=True)
)
x3a += (3.0/12.0) * (
        -0.5*np.einsum('mnIJ,BAcmnK->ABcIJK', H.aa.oooo[oa, oa, Oa, Oa], T.aaa.VVvooO, optimize=True)
        -1.0*np.einsum('mNIJ,BAcmNK->ABcIJK', H.aa.oooo[oa, Oa, Oa, Oa], T.aaa.VVvoOO, optimize=True)
        -0.5*np.einsum('MNIJ,BAcMNK->ABcIJK', H.aa.oooo[Oa, Oa, Oa, Oa], T.aaa.VVvOOO, optimize=True)
)
x3a += (1.0/12.0) * (
        +1.0*np.einsum('ABeF,FceIJK->ABcIJK', H.aa.vvvv[Va, Va, va, Va], T.aaa.VvvOOO, optimize=True)
        -0.5*np.einsum('ABEF,FEcIJK->ABcIJK', H.aa.vvvv[Va, Va, Va, Va], T.aaa.VVvOOO, optimize=True)
)
x3a += (2.0/12.0) * (
        +0.5*np.einsum('cBef,AfeIJK->ABcIJK', H.aa.vvvv[va, Va, va, va], T.aaa.VvvOOO, optimize=True)
        +1.0*np.einsum('cBeF,AFeIJK->ABcIJK', H.aa.vvvv[va, Va, va, Va], T.aaa.VVvOOO, optimize=True)
        +0.5*np.einsum('cBEF,AFEIJK->ABcIJK', H.aa.vvvv[va, Va, Va, Va], T.aaa.VVVOOO, optimize=True)
)
x3a += (6.0/12.0) * (
        +1.0*np.einsum('AmIe,BcemJK->ABcIJK', H.aa.voov[Va, oa, Oa, va], T.aaa.VvvoOO, optimize=True)
        -1.0*np.einsum('AmIE,BEcmJK->ABcIJK', H.aa.voov[Va, oa, Oa, Va], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('AMIe,BceMJK->ABcIJK', H.aa.voov[Va, Oa, Oa, va], T.aaa.VvvOOO, optimize=True)
        -1.0*np.einsum('AMIE,BEcMJK->ABcIJK', H.aa.voov[Va, Oa, Oa, Va], T.aaa.VVvOOO, optimize=True)
)
x3a += (3.0/12.0) * (
        +1.0*np.einsum('cmIe,ABemJK->ABcIJK', H.aa.voov[va, oa, Oa, va], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('cmIE,ABEmJK->ABcIJK', H.aa.voov[va, oa, Oa, Va], T.aaa.VVVoOO, optimize=True)
        +1.0*np.einsum('cMIe,ABeMJK->ABcIJK', H.aa.voov[va, Oa, Oa, va], T.aaa.VVvOOO, optimize=True)
        +1.0*np.einsum('cMIE,ABEMJK->ABcIJK', H.aa.voov[va, Oa, Oa, Va], T.aaa.VVVOOO, optimize=True)
)
x3a += (6.0/12.0) * (
        +1.0*np.einsum('AmIe,BceJKm->ABcIJK', H.ab.voov[Va, ob, Oa, vb], T.aab.VvvOOo, optimize=True)
        +1.0*np.einsum('AmIE,BcEJKm->ABcIJK', H.ab.voov[Va, ob, Oa, Vb], T.aab.VvVOOo, optimize=True)
        +1.0*np.einsum('AMIe,BceJKM->ABcIJK', H.ab.voov[Va, Ob, Oa, vb], T.aab.VvvOOO, optimize=True)
        +1.0*np.einsum('AMIE,BcEJKM->ABcIJK', H.ab.voov[Va, Ob, Oa, Vb], T.aab.VvVOOO, optimize=True)
)
x3a += (3.0/12.0) * (
        +1.0*np.einsum('cmIe,ABeJKm->ABcIJK', H.ab.voov[va, ob, Oa, vb], T.aab.VVvOOo, optimize=True)
        +1.0*np.einsum('cmIE,ABEJKm->ABcIJK', H.ab.voov[va, ob, Oa, Vb], T.aab.VVVOOo, optimize=True)
        +1.0*np.einsum('cMIe,ABeJKM->ABcIJK', H.ab.voov[va, Ob, Oa, vb], T.aab.VVvOOO, optimize=True)
        +1.0*np.einsum('cMIE,ABEJKM->ABcIJK', H.ab.voov[va, Ob, Oa, Vb], T.aab.VVVOOO, optimize=True)
)


x3a -= np.transpose(x3a, (1, 0, 2, 3, 4, 5))

x3a -= np.transpose(x3a, (0, 1, 2, 4, 3, 5)) + np.transpose(x3a, (0, 1, 2, 3, 5, 4))\
     + np.transpose(x3a, (0, 1, 2, 5, 4, 3)) - np.transpose(x3a, (0, 1, 2, 4, 5, 3))\
     - np.transpose(x3a, (0, 1, 2, 5, 3, 4))
        
get_error(X3A[Va, Va, va, Oa, Oa, Oa], x3a)

1.7988592831819637e-18


### Projection 3

In [13]:
x3a = (1.0/12.0) * (
        +1.0*np.einsum('mi,CBAmJK->ABCiJK', H.a.oo[oa, oa], T.aaa.VVVoOO, optimize=True)
        +1.0*np.einsum('Mi,CBAMJK->ABCiJK', H.a.oo[Oa, oa], T.aaa.VVVOOO, optimize=True)
)
x3a += (2.0/12.0) * (
        -1.0*np.einsum('mJ,CBAmiK->ABCiJK', H.a.oo[oa, Oa], T.aaa.VVVooO, optimize=True)
        +1.0*np.einsum('MJ,CBAiMK->ABCiJK', H.a.oo[Oa, Oa], T.aaa.VVVoOO, optimize=True)
)

x3a += (3.0/12.0) * (
        -1.0*np.einsum('Ae,CBeiJK->ABCiJK', H.a.vv[Va, va], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('AE,CBEiJK->ABCiJK', H.a.vv[Va, Va], T.aaa.VVVoOO, optimize=True)
)

x3a += (2.0/12.0) * (
        -0.5*np.einsum('mniJ,CBAmnK->ABCiJK', H.aa.oooo[oa, oa, oa, Oa], T.aaa.VVVooO, optimize=True)
        +1.0*np.einsum('MniJ,CBAnMK->ABCiJK', H.aa.oooo[Oa, oa, oa, Oa], T.aaa.VVVoOO, optimize=True)
        -0.5*np.einsum('MNiJ,CBAMNK->ABCiJK', H.aa.oooo[Oa, Oa, oa, Oa], T.aaa.VVVOOO, optimize=True)
)
x3a += (1.0/12.0) * (
        +1.0*np.einsum('MnKJ,CBAniM->ABCiJK', H.aa.oooo[Oa, oa, Oa, Oa], T.aaa.VVVooO, optimize=True)
        +0.5*np.einsum('MNKJ,CBAiMN->ABCiJK', H.aa.oooo[Oa, Oa, Oa, Oa], T.aaa.VVVoOO, optimize=True)
)

x3a += (3.0/12.0) * (
        -0.5*np.einsum('ABef,CfeiJK->ABCiJK', H.aa.vvvv[Va, Va, va, va], T.aaa.VvvoOO, optimize=True)
        +1.0*np.einsum('ABEf,CEfiJK->ABCiJK', H.aa.vvvv[Va, Va, Va, va], T.aaa.VVvoOO, optimize=True)
        -0.5*np.einsum('ABEF,CFEiJK->ABCiJK', H.aa.vvvv[Va, Va, Va, Va], T.aaa.VVVoOO, optimize=True)
)

x3a += (3.0/12.0) * (
        -1.0*np.einsum('Amie,CBemJK->ABCiJK', H.aa.voov[Va, oa, oa, va], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('AMie,CBeMJK->ABCiJK', H.aa.voov[Va, Oa, oa, va], T.aaa.VVvOOO, optimize=True)
        -1.0*np.einsum('AmiE,CBEmJK->ABCiJK', H.aa.voov[Va, oa, oa, Va], T.aaa.VVVoOO, optimize=True)
        -1.0*np.einsum('AMiE,CBEMJK->ABCiJK', H.aa.voov[Va, Oa, oa, Va], T.aaa.VVVOOO, optimize=True)
)
x3a += (6.0/12.0) * (
        +1.0*np.einsum('AmJe,CBemiK->ABCiJK', H.aa.voov[Va, oa, Oa, va], T.aaa.VVvooO, optimize=True)
        -1.0*np.einsum('AMJe,CBeiMK->ABCiJK', H.aa.voov[Va, Oa, Oa, va], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('AmJE,CBEmiK->ABCiJK', H.aa.voov[Va, oa, Oa, Va], T.aaa.VVVooO, optimize=True)
        -1.0*np.einsum('AMJE,CBEiMK->ABCiJK', H.aa.voov[Va, Oa, Oa, Va], T.aaa.VVVoOO, optimize=True)
)

x3a += (3.0/12.0) * (
        -1.0*np.einsum('Amie,CBeJKm->ABCiJK', H.ab.voov[Va, ob, oa, vb], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('AMie,CBeJKM->ABCiJK', H.ab.voov[Va, Ob, oa, vb], T.aab.VVvOOO, optimize=True)
        -1.0*np.einsum('AmiE,CBEJKm->ABCiJK', H.ab.voov[Va, ob, oa, Vb], T.aab.VVVOOo, optimize=True)
        -1.0*np.einsum('AMiE,CBEJKM->ABCiJK', H.ab.voov[Va, Ob, oa, Vb], T.aab.VVVOOO, optimize=True)
)
x3a += (6.0/12.0) * (
        +1.0*np.einsum('AmJe,CBeiKm->ABCiJK', H.ab.voov[Va, ob, Oa, vb], T.aab.VVvoOo, optimize=True)
        +1.0*np.einsum('AMJe,CBeiKM->ABCiJK', H.ab.voov[Va, Ob, Oa, vb], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('AmJE,CBEiKm->ABCiJK', H.ab.voov[Va, ob, Oa, Vb], T.aab.VVVoOo, optimize=True)
        +1.0*np.einsum('AMJE,CBEiKM->ABCiJK', H.ab.voov[Va, Ob, Oa, Vb], T.aab.VVVoOO, optimize=True)
)


x3a -= np.transpose(x3a, (1, 0, 2, 3, 4, 5)) + np.transpose(x3a, (0, 2, 1, 3, 4, 5))\
     + np.transpose(x3a, (2, 1, 0, 3, 4, 5)) - np.transpose(x3a, (1, 2, 0, 3, 4, 5))\
     - np.transpose(x3a, (2, 0, 1, 3, 4, 5))

x3a -= np.transpose(x3a, (0, 1, 2, 3, 5, 4))

get_error(X3A[Va, Va, Va, oa, Oa, Oa], x3a)

1.2947760209731236e-17


### Projection 4

In [14]:

x3a = (1.0/4.0) * (
        +1.0*np.einsum('mi,BAcmJK->ABciJK', H.a.oo[oa, oa], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('Mi,BAcMJK->ABciJK', H.a.oo[Oa, oa], T.aaa.VVvOOO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('mJ,BAcmiK->ABciJK', H.a.oo[oa, Oa], T.aaa.VVvooO, optimize=True)
        +1.0*np.einsum('MJ,BAciMK->ABciJK', H.a.oo[Oa, Oa], T.aaa.VVvoOO, optimize=True)
)

x3a += (2.0/4.0) * (
        +1.0*np.einsum('Ae,BceiJK->ABciJK', H.a.vv[Va, va], T.aaa.VvvoOO, optimize=True)
        -1.0*np.einsum('AE,BEciJK->ABciJK', H.a.vv[Va, Va], T.aaa.VVvoOO, optimize=True)
)
x3a += (1.0/4.0) * (
        +1.0*np.einsum('ce,ABeiJK->ABciJK', H.a.vv[va, va], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('cE,ABEiJK->ABciJK', H.a.vv[va, Va], T.aaa.VVVoOO, optimize=True)
)

x3a += (2.0/4.0) * (
        -0.5*np.einsum('mniJ,BAcmnK->ABciJK', H.aa.oooo[oa, oa, oa, Oa], T.aaa.VVvooO, optimize=True)
        -1.0*np.einsum('mNiJ,BAcmNK->ABciJK', H.aa.oooo[oa, Oa, oa, Oa], T.aaa.VVvoOO, optimize=True)
        -0.5*np.einsum('MNiJ,BAcMNK->ABciJK', H.aa.oooo[Oa, Oa, oa, Oa], T.aaa.VVvOOO, optimize=True)
)
x3a += (1.0/4.0) * (
        -1.0*np.einsum('mNKJ,BAcmiN->ABciJK', H.aa.oooo[oa, Oa, Oa, Oa], T.aaa.VVvooO, optimize=True)
        +0.5*np.einsum('MNKJ,BAciMN->ABciJK', H.aa.oooo[Oa, Oa, Oa, Oa], T.aaa.VVvoOO, optimize=True)
)

x3a += (1.0/4.0) * (
        -1.0*np.einsum('ABEf,EcfiJK->ABciJK', H.aa.vvvv[Va, Va, Va, va], T.aaa.VvvoOO, optimize=True)
        -0.5*np.einsum('ABEF,FEciJK->ABciJK', H.aa.vvvv[Va, Va, Va, Va], T.aaa.VVvoOO, optimize=True)
)
x3a += (2.0/4.0) * (
        +0.5*np.einsum('cBef,AfeiJK->ABciJK', H.aa.vvvv[va, Va, va, va], T.aaa.VvvoOO, optimize=True)
        -1.0*np.einsum('cBEf,AEfiJK->ABciJK', H.aa.vvvv[va, Va, Va, va], T.aaa.VVvoOO, optimize=True)
        +0.5*np.einsum('cBEF,AFEiJK->ABciJK', H.aa.vvvv[va, Va, Va, Va], T.aaa.VVVoOO, optimize=True)
)

x3a += (2.0/4.0) * (
        +1.0*np.einsum('Amie,BcemJK->ABciJK', H.aa.voov[Va, oa, oa, va], T.aaa.VvvoOO, optimize=True)
        +1.0*np.einsum('AMie,BceMJK->ABciJK', H.aa.voov[Va, Oa, oa, va], T.aaa.VvvOOO, optimize=True)
        -1.0*np.einsum('AmiE,BEcmJK->ABciJK', H.aa.voov[Va, oa, oa, Va], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('AMiE,BEcMJK->ABciJK', H.aa.voov[Va, Oa, oa, Va], T.aaa.VVvOOO, optimize=True)
)
x3a += (1.0/4.0) * (
        +1.0*np.einsum('cmie,ABemJK->ABciJK', H.aa.voov[va, oa, oa, va], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('cMie,ABeMJK->ABciJK', H.aa.voov[va, Oa, oa, va], T.aaa.VVvOOO, optimize=True)
        +1.0*np.einsum('cmiE,ABEmJK->ABciJK', H.aa.voov[va, oa, oa, Va], T.aaa.VVVoOO, optimize=True)
        +1.0*np.einsum('cMiE,ABEMJK->ABciJK', H.aa.voov[va, Oa, oa, Va], T.aaa.VVVOOO, optimize=True)
)
x3a += (4.0/4.0) * (
        -1.0*np.einsum('AmJe,BcemiK->ABciJK', H.aa.voov[Va, oa, Oa, va], T.aaa.VvvooO, optimize=True)
        +1.0*np.einsum('AMJe,BceiMK->ABciJK', H.aa.voov[Va, Oa, Oa, va], T.aaa.VvvoOO, optimize=True)
        +1.0*np.einsum('AmJE,BEcmiK->ABciJK', H.aa.voov[Va, oa, Oa, Va], T.aaa.VVvooO, optimize=True)
        -1.0*np.einsum('AMJE,BEciMK->ABciJK', H.aa.voov[Va, Oa, Oa, Va], T.aaa.VVvoOO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('cmJe,ABemiK->ABciJK', H.aa.voov[va, oa, Oa, va], T.aaa.VVvooO, optimize=True)
        +1.0*np.einsum('cMJe,ABeiMK->ABciJK', H.aa.voov[va, Oa, Oa, va], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('cmJE,ABEmiK->ABciJK', H.aa.voov[va, oa, Oa, Va], T.aaa.VVVooO, optimize=True)
        +1.0*np.einsum('cMJE,ABEiMK->ABciJK', H.aa.voov[va, Oa, Oa, Va], T.aaa.VVVoOO, optimize=True)
)

x3a += (2.0/4.0) * (
        +1.0*np.einsum('Amie,BceJKm->ABciJK', H.ab.voov[Va, ob, oa, vb], T.aab.VvvOOo, optimize=True)
        +1.0*np.einsum('AMie,BceJKM->ABciJK', H.ab.voov[Va, Ob, oa, vb], T.aab.VvvOOO, optimize=True)
        +1.0*np.einsum('AmiE,BcEJKm->ABciJK', H.ab.voov[Va, ob, oa, Vb], T.aab.VvVOOo, optimize=True)
        +1.0*np.einsum('AMiE,BcEJKM->ABciJK', H.ab.voov[Va, Ob, oa, Vb], T.aab.VvVOOO, optimize=True)
)
x3a += (1.0/4.0) * (
        +1.0*np.einsum('cmie,ABeJKm->ABciJK', H.ab.voov[va, ob, oa, vb], T.aab.VVvOOo, optimize=True)
        +1.0*np.einsum('cMie,ABeJKM->ABciJK', H.ab.voov[va, Ob, oa, vb], T.aab.VVvOOO, optimize=True)
        +1.0*np.einsum('cmiE,ABEJKm->ABciJK', H.ab.voov[va, ob, oa, Vb], T.aab.VVVOOo, optimize=True)
        +1.0*np.einsum('cMiE,ABEJKM->ABciJK', H.ab.voov[va, Ob, oa, Vb], T.aab.VVVOOO, optimize=True)
)
x3a += (4.0/4.0) * (
        -1.0*np.einsum('AmJe,BceiKm->ABciJK', H.ab.voov[Va, ob, Oa, vb], T.aab.VvvoOo, optimize=True)
        -1.0*np.einsum('AMJe,BceiKM->ABciJK', H.ab.voov[Va, Ob, Oa, vb], T.aab.VvvoOO, optimize=True)
        -1.0*np.einsum('AmJE,BcEiKm->ABciJK', H.ab.voov[Va, ob, Oa, Vb], T.aab.VvVoOo, optimize=True)
        -1.0*np.einsum('AMJE,BcEiKM->ABciJK', H.ab.voov[Va, Ob, Oa, Vb], T.aab.VvVoOO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('cmJe,ABeiKm->ABciJK', H.ab.voov[va, ob, Oa, vb], T.aab.VVvoOo, optimize=True)
        -1.0*np.einsum('cMJe,ABeiKM->ABciJK', H.ab.voov[va, Ob, Oa, vb], T.aab.VVvoOO, optimize=True)
        -1.0*np.einsum('cmJE,ABEiKm->ABciJK', H.ab.voov[va, ob, Oa, Vb], T.aab.VVVoOo, optimize=True)
        -1.0*np.einsum('cMJE,ABEiKM->ABciJK', H.ab.voov[va, Ob, Oa, Vb], T.aab.VVVoOO, optimize=True)
)

x3a -= np.transpose(x3a, (1, 0, 2, 3, 4, 5))
x3a -= np.transpose(x3a, (0, 1, 2, 3, 5, 4))

get_error(X3A[Va, Va, va, oa, Oa, Oa], x3a)

6.374866167319049e-18


### Projection 5

In [15]:
x3a = (3.0/12.0) * (
        +1.0*np.einsum('mI,AcbmJK->AbcIJK', H.a.oo[oa, Oa], T.aaa.VvvoOO, optimize=True)
        +1.0*np.einsum('MI,AcbMJK->AbcIJK', H.a.oo[Oa, Oa], T.aaa.VvvOOO, optimize=True)
)
x3a += (1.0/12.0) * (
        -1.0*np.einsum('AE,EcbIJK->AbcIJK', H.a.vv[Va, Va], T.aaa.VvvOOO, optimize=True)
)
x3a += (2.0/12.0) * (
        -1.0*np.einsum('be,AceIJK->AbcIJK', H.a.vv[va, va], T.aaa.VvvOOO, optimize=True)
        +1.0*np.einsum('bE,AEcIJK->AbcIJK', H.a.vv[va, Va], T.aaa.VVvOOO, optimize=True)
)
x3a += (3.0/12.0) * (
        -0.5*np.einsum('mnIJ,AcbmnK->AbcIJK', H.aa.oooo[oa, oa, Oa, Oa], T.aaa.VvvooO, optimize=True)
        -1.0*np.einsum('mNIJ,AcbmNK->AbcIJK', H.aa.oooo[oa, Oa, Oa, Oa], T.aaa.VvvoOO, optimize=True)
        -0.5*np.einsum('MNIJ,AcbMNK->AbcIJK', H.aa.oooo[Oa, Oa, Oa, Oa], T.aaa.VvvOOO, optimize=True)
)
x3a += (2.0/12.0) * (
        -1.0*np.einsum('AbEf,EcfIJK->AbcIJK', H.aa.vvvv[Va, va, Va, va], T.aaa.VvvOOO, optimize=True)
        -0.5*np.einsum('AbEF,FEcIJK->AbcIJK', H.aa.vvvv[Va, va, Va, Va], T.aaa.VVvOOO, optimize=True)
)
x3a += (1.0/12.0) * (
        +0.5*np.einsum('cbef,AfeIJK->AbcIJK', H.aa.vvvv[va, va, va, va], T.aaa.VvvOOO, optimize=True)
        -1.0*np.einsum('cbEf,AEfIJK->AbcIJK', H.aa.vvvv[va, va, Va, va], T.aaa.VVvOOO, optimize=True)
        +0.5*np.einsum('cbEF,AFEIJK->AbcIJK', H.aa.vvvv[va, va, Va, Va], T.aaa.VVVOOO, optimize=True)
)
x3a += (3.0/12.0) * (
        -1.0*np.einsum('AmIE,EcbmJK->AbcIJK', H.aa.voov[Va, oa, Oa, Va], T.aaa.VvvoOO, optimize=True)
        -1.0*np.einsum('AMIE,EcbMJK->AbcIJK', H.aa.voov[Va, Oa, Oa, Va], T.aaa.VvvOOO, optimize=True)
)
x3a += (6.0/12.0) * (
        -1.0*np.einsum('bmIe,AcemJK->AbcIJK', H.aa.voov[va, oa, Oa, va], T.aaa.VvvoOO, optimize=True)
        +1.0*np.einsum('bmIE,AEcmJK->AbcIJK', H.aa.voov[va, oa, Oa, Va], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('bMIe,AceMJK->AbcIJK', H.aa.voov[va, Oa, Oa, va], T.aaa.VvvOOO, optimize=True)
        +1.0*np.einsum('bMIE,AEcMJK->AbcIJK', H.aa.voov[va, Oa, Oa, Va], T.aaa.VVvOOO, optimize=True)
)
x3a += (3.0/12.0) * (
        -1.0*np.einsum('AmIE,cbEJKm->AbcIJK', H.ab.voov[Va, ob, Oa, Vb], T.aab.vvVOOo, optimize=True)
        -1.0*np.einsum('AMIE,cbEJKM->AbcIJK', H.ab.voov[Va, Ob, Oa, Vb], T.aab.vvVOOO, optimize=True)
)
x3a += (6.0/12.0) * (
        -1.0*np.einsum('bmIe,AceJKm->AbcIJK', H.ab.voov[va, ob, Oa, vb], T.aab.VvvOOo, optimize=True)
        -1.0*np.einsum('bmIE,AcEJKm->AbcIJK', H.ab.voov[va, ob, Oa, Vb], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('bMIe,AceJKM->AbcIJK', H.ab.voov[va, Ob, Oa, vb], T.aab.VvvOOO, optimize=True)
        -1.0*np.einsum('bMIE,AcEJKM->AbcIJK', H.ab.voov[va, Ob, Oa, Vb], T.aab.VvVOOO, optimize=True)
)

x3a -= np.transpose(x3a, (0, 1, 2, 4, 3, 5)) + np.transpose(x3a, (0, 1, 2, 3, 5, 4))\
     + np.transpose(x3a, (0, 1, 2, 5, 4, 3)) - np.transpose(x3a, (0, 1, 2, 4, 5, 3))\
     - np.transpose(x3a, (0, 1, 2, 5, 3, 4))

x3a -= np.transpose(x3a, (0, 2, 1, 3, 4, 5))
        
get_error(X3A[Va, va, va, Oa, Oa, Oa], x3a)

3.543851891868178e-18


### Projection 6

In [16]:
x3a = (2.0/12.0) * (
        +1.0*np.einsum('mi,CBAmjK->ABCijK', H.a.oo[oa, oa], T.aaa.VVVooO, optimize=True)
        -1.0*np.einsum('Mi,CBAjMK->ABCijK', H.a.oo[Oa, oa], T.aaa.VVVoOO, optimize=True)
)
x3a += (1.0/12.0) * (
        -1.0*np.einsum('MK,CBAjiM->ABCijK', H.a.oo[Oa, Oa], T.aaa.VVVooO, optimize=True)
)
x3a += (3.0/12.0) * (
        -1.0*np.einsum('Ae,CBeijK->ABCijK', H.a.vv[Va, va], T.aaa.VVvooO, optimize=True)
        -1.0*np.einsum('AE,CBEijK->ABCijK', H.a.vv[Va, Va], T.aaa.VVVooO, optimize=True)
)
x3a += (1.0/12.0) * (
        -0.5*np.einsum('mnij,CBAmnK->ABCijK', H.aa.oooo[oa, oa, oa, oa], T.aaa.VVVooO, optimize=True)
        +1.0*np.einsum('Mnij,CBAnMK->ABCijK', H.aa.oooo[Oa, oa, oa, oa], T.aaa.VVVoOO, optimize=True)
        -0.5*np.einsum('MNij,CBAMNK->ABCijK', H.aa.oooo[Oa, Oa, oa, oa], T.aaa.VVVOOO, optimize=True)
)
x3a += (2.0/12.0) * (
        +1.0*np.einsum('MnKj,CBAniM->ABCijK', H.aa.oooo[Oa, oa, Oa, oa], T.aaa.VVVooO, optimize=True)
        +0.5*np.einsum('MNKj,CBAiMN->ABCijK', H.aa.oooo[Oa, Oa, Oa, oa], T.aaa.VVVoOO, optimize=True)
)
x3a += (3.0/12.0) * (
        -0.5*np.einsum('ABef,CfeijK->ABCijK', H.aa.vvvv[Va, Va, va, va], T.aaa.VvvooO, optimize=True)
        -1.0*np.einsum('ABeF,CFeijK->ABCijK', H.aa.vvvv[Va, Va, va, Va], T.aaa.VVvooO, optimize=True)
        -0.5*np.einsum('ABEF,CFEijK->ABCijK', H.aa.vvvv[Va, Va, Va, Va], T.aaa.VVVooO, optimize=True)
)
x3a += (6.0/12.0) * (
        -1.0*np.einsum('Amie,CBemjK->ABCijK', H.aa.voov[Va, oa, oa, va], T.aaa.VVvooO, optimize=True)
        -1.0*np.einsum('AmiE,CBEmjK->ABCijK', H.aa.voov[Va, oa, oa, Va], T.aaa.VVVooO, optimize=True)
        +1.0*np.einsum('AMie,CBejMK->ABCijK', H.aa.voov[Va, Oa, oa, va], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('AMiE,CBEjMK->ABCijK', H.aa.voov[Va, Oa, oa, Va], T.aaa.VVVoOO, optimize=True)
)
x3a += (3.0/12.0) * (
        +1.0*np.einsum('AMKe,CBejiM->ABCijK', H.aa.voov[Va, Oa, Oa, va], T.aaa.VVvooO, optimize=True)
        +1.0*np.einsum('AMKE,CBEjiM->ABCijK', H.aa.voov[Va, Oa, Oa, Va], T.aaa.VVVooO, optimize=True)
)

### PROBLEM ###
x3a += (6.0/12.0) * (
        -1.0*np.einsum('Amie,CBejKm->ABCijK', H.ab.voov[Va, ob, oa, vb], T.aab.VVvoOo, optimize=True)
        -1.0*np.einsum('AmiE,CBEjKm->ABCijK', H.ab.voov[Va, ob, oa, Vb], T.aab.VVVoOo, optimize=True)
        -1.0*np.einsum('AMie,CBejKM->ABCijK', H.ab.voov[Va, Ob, oa, vb], T.aab.VVvoOO, optimize=True)
        -1.0*np.einsum('AMiE,CBEjKM->ABCijK', H.ab.voov[Va, Ob, oa, Vb], T.aab.VVVoOO, optimize=True)
)
x3a += (3.0/12.0) * (
        +1.0*np.einsum('AMKe,CBejiM->ABCijK', H.ab.voov[Va, Ob, Oa, vb], T.aab.VVvooO, optimize=True)
        +1.0*np.einsum('AMKE,CBEjiM->ABCijK', H.ab.voov[Va, Ob, Oa, Vb], T.aab.VVVooO, optimize=True)
)

x3a -= np.transpose(x3a, (1, 0, 2, 3, 4, 5)) + np.transpose(x3a, (0, 2, 1, 3, 4, 5))\
     + np.transpose(x3a, (2, 1, 0, 3, 4, 5)) - np.transpose(x3a, (1, 2, 0, 3, 4, 5))\
     - np.transpose(x3a, (2, 0, 1, 3, 4, 5))

x3a -= np.transpose(x3a, (0, 1, 2, 4, 3, 5))

get_error(X3A[Va, Va, Va, oa, oa, Oa], x3a)

6.525704725415481e-18


### Projection 7

In [17]:
x3a = (2.0/4.0) * (
        +1.0*np.einsum('mi,BAcmjK->ABcijK', H.a.oo[oa, oa], T.aaa.VVvooO, optimize=True)
        -1.0*np.einsum('Mi,BAcjMK->ABcijK', H.a.oo[Oa, oa], T.aaa.VVvoOO, optimize=True)
)
x3a += (1.0/4.0) * (
        -1.0*np.einsum('MK,BAcjiM->ABcijK', H.a.oo[Oa, Oa], T.aaa.VVvooO, optimize=True)
)
x3a += (2.0/4.0) * (
        +1.0*np.einsum('Ae,BceijK->ABcijK', H.a.vv[Va, va], T.aaa.VvvooO, optimize=True)
        -1.0*np.einsum('AE,BEcijK->ABcijK', H.a.vv[Va, Va], T.aaa.VVvooO, optimize=True)
)
x3a += (1.0/4.0) * (
        +1.0*np.einsum('ce,ABeijK->ABcijK', H.a.vv[va, va], T.aaa.VVvooO, optimize=True)
        +1.0*np.einsum('cE,ABEijK->ABcijK', H.a.vv[va, Va], T.aaa.VVVooO, optimize=True)
)
x3a += (1.0/4.0) * (
        -0.5*np.einsum('mnij,BAcmnK->ABcijK', H.aa.oooo[oa, oa, oa, oa], T.aaa.VVvooO, optimize=True)
        -1.0*np.einsum('mNij,BAcmNK->ABcijK', H.aa.oooo[oa, Oa, oa, oa], T.aaa.VVvoOO, optimize=True)
        -0.5*np.einsum('MNij,BAcMNK->ABcijK', H.aa.oooo[Oa, Oa, oa, oa], T.aaa.VVvOOO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('mNKj,BAcmiN->ABcijK', H.aa.oooo[oa, Oa, Oa, oa], T.aaa.VVvooO, optimize=True)
        +0.5*np.einsum('MNKj,BAciMN->ABcijK', H.aa.oooo[Oa, Oa, Oa, oa], T.aaa.VVvoOO, optimize=True)
)
x3a += (1.0/4.0) * (
        +1.0*np.einsum('ABeF,FceijK->ABcijK', H.aa.vvvv[Va, Va, va, Va], T.aaa.VvvooO, optimize=True)
        -0.5*np.einsum('ABEF,FEcijK->ABcijK', H.aa.vvvv[Va, Va, Va, Va], T.aaa.VVvooO, optimize=True)
)
x3a += (2.0/4.0) * (
        +0.5*np.einsum('cBef,AfeijK->ABcijK', H.aa.vvvv[va, Va, va, va], T.aaa.VvvooO, optimize=True)
        +1.0*np.einsum('cBeF,AFeijK->ABcijK', H.aa.vvvv[va, Va, va, Va], T.aaa.VVvooO, optimize=True)
        +0.5*np.einsum('cBEF,AFEijK->ABcijK', H.aa.vvvv[va, Va, Va, Va], T.aaa.VVVooO, optimize=True)
)
x3a += (4.0/4.0) * (
        +1.0*np.einsum('Amie,BcemjK->ABcijK', H.aa.voov[Va, oa, oa, va], T.aaa.VvvooO, optimize=True)
        -1.0*np.einsum('AmiE,BEcmjK->ABcijK', H.aa.voov[Va, oa, oa, Va], T.aaa.VVvooO, optimize=True)
        -1.0*np.einsum('AMie,BcejMK->ABcijK', H.aa.voov[Va, Oa, oa, va], T.aaa.VvvoOO, optimize=True)
        +1.0*np.einsum('AMiE,BEcjMK->ABcijK', H.aa.voov[Va, Oa, oa, Va], T.aaa.VVvoOO, optimize=True)
)
x3a += (2.0/4.0) * (
        +1.0*np.einsum('cmie,ABemjK->ABcijK', H.aa.voov[va, oa, oa, va], T.aaa.VVvooO, optimize=True)
        +1.0*np.einsum('cmiE,ABEmjK->ABcijK', H.aa.voov[va, oa, oa, Va], T.aaa.VVVooO, optimize=True)
        -1.0*np.einsum('cMie,ABejMK->ABcijK', H.aa.voov[va, Oa, oa, va], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('cMiE,ABEjMK->ABcijK', H.aa.voov[va, Oa, oa, Va], T.aaa.VVVoOO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('AMKe,BcejiM->ABcijK', H.aa.voov[Va, Oa, Oa, va], T.aaa.VvvooO, optimize=True)
        +1.0*np.einsum('AMKE,BEcjiM->ABcijK', H.aa.voov[Va, Oa, Oa, Va], T.aaa.VVvooO, optimize=True)
)
x3a += (1.0/4.0) * (
        -1.0*np.einsum('cMKe,ABejiM->ABcijK', H.aa.voov[va, Oa, Oa, va], T.aaa.VVvooO, optimize=True)
        -1.0*np.einsum('cMKE,ABEjiM->ABcijK', H.aa.voov[va, Oa, Oa, Va], T.aaa.VVVooO, optimize=True)
)

x3a += (4.0/4.0) * (
        +1.0*np.einsum('Amie,BcejKm->ABcijK', H.ab.voov[Va, ob, oa, vb], T.aab.VvvoOo, optimize=True)
        +1.0*np.einsum('AmiE,BcEjKm->ABcijK', H.ab.voov[Va, ob, oa, Vb], T.aab.VvVoOo, optimize=True)
        +1.0*np.einsum('AMie,BcejKM->ABcijK', H.ab.voov[Va, Ob, oa, vb], T.aab.VvvoOO, optimize=True)
        +1.0*np.einsum('AMiE,BcEjKM->ABcijK', H.ab.voov[Va, Ob, oa, Vb], T.aab.VvVoOO, optimize=True)
)
x3a += (2.0/4.0) * (
        +1.0*np.einsum('cmie,ABejKm->ABcijK', H.ab.voov[va, ob, oa, vb], T.aab.VVvoOo, optimize=True)
        +1.0*np.einsum('cmiE,ABEjKm->ABcijK', H.ab.voov[va, ob, oa, Vb], T.aab.VVVoOo, optimize=True)
        +1.0*np.einsum('cMie,ABejKM->ABcijK', H.ab.voov[va, Ob, oa, vb], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('cMiE,ABEjKM->ABcijK', H.ab.voov[va, Ob, oa, Vb], T.aab.VVVoOO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('AMKe,BcejiM->ABcijK', H.ab.voov[Va, Ob, Oa, vb], T.aab.VvvooO, optimize=True)
        -1.0*np.einsum('AMKE,BcEjiM->ABcijK', H.ab.voov[Va, Ob, Oa, Vb], T.aab.VvVooO, optimize=True)
)
x3a += (1.0/4.0) * (
        -1.0*np.einsum('cMKe,ABejiM->ABcijK', H.ab.voov[va, Ob, Oa, vb], T.aab.VVvooO, optimize=True)
        -1.0*np.einsum('cMKE,ABEjiM->ABcijK', H.ab.voov[va, Ob, Oa, Vb], T.aab.VVVooO, optimize=True)
)


x3a -= np.transpose(x3a, (1, 0, 2, 3, 4, 5))
x3a -= np.transpose(x3a, (0, 1, 2, 4, 3, 5))

get_error(X3A[Va, Va, va, oa, oa, Oa], x3a)

4.105170594651676e-18


### Projection 8

In [18]:
x3a = (1.0/4.0) * (
        +1.0*np.einsum('mi,AcbmJK->AbciJK', H.a.oo[oa, oa], T.aaa.VvvoOO, optimize=True)
        +1.0*np.einsum('Mi,AcbMJK->AbciJK', H.a.oo[Oa, oa], T.aaa.VvvOOO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('mJ,AcbmiK->AbciJK', H.a.oo[oa, Oa], T.aaa.VvvooO, optimize=True)
        +1.0*np.einsum('MJ,AcbiMK->AbciJK', H.a.oo[Oa, Oa], T.aaa.VvvoOO, optimize=True)
)
x3a += (1.0/4.0) * (
        -1.0*np.einsum('AE,EcbiJK->AbciJK', H.a.vv[Va, Va], T.aaa.VvvoOO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('be,AceiJK->AbciJK', H.a.vv[va, va], T.aaa.VvvoOO, optimize=True)
        +1.0*np.einsum('bE,AEciJK->AbciJK', H.a.vv[va, Va], T.aaa.VVvoOO, optimize=True)
)
x3a += (2.0/4.0) * (
        -0.5*np.einsum('mniJ,AcbmnK->AbciJK', H.aa.oooo[oa, oa, oa, Oa], T.aaa.VvvooO, optimize=True)
        -1.0*np.einsum('mNiJ,AcbmNK->AbciJK', H.aa.oooo[oa, Oa, oa, Oa], T.aaa.VvvoOO, optimize=True)
        -0.5*np.einsum('MNiJ,AcbMNK->AbciJK', H.aa.oooo[Oa, Oa, oa, Oa], T.aaa.VvvOOO, optimize=True)
)
x3a += (1.0/4.0) * (
        -1.0*np.einsum('mNKJ,AcbmiN->AbciJK', H.aa.oooo[oa, Oa, Oa, Oa], T.aaa.VvvooO, optimize=True)
        +0.5*np.einsum('MNKJ,AcbiMN->AbciJK', H.aa.oooo[Oa, Oa, Oa, Oa], T.aaa.VvvoOO, optimize=True)
)
x3a += (2.0/4.0) * (
        +1.0*np.einsum('AbeF,FceiJK->AbciJK', H.aa.vvvv[Va, va, va, Va], T.aaa.VvvoOO, optimize=True)
        -0.5*np.einsum('AbEF,FEciJK->AbciJK', H.aa.vvvv[Va, va, Va, Va], T.aaa.VVvoOO, optimize=True)
)
x3a += (1.0/4.0) * (
        +0.5*np.einsum('cbef,AfeiJK->AbciJK', H.aa.vvvv[va, va, va, va], T.aaa.VvvoOO, optimize=True)
        +1.0*np.einsum('cbeF,AFeiJK->AbciJK', H.aa.vvvv[va, va, va, Va], T.aaa.VVvoOO, optimize=True)
        +0.5*np.einsum('cbEF,AFEiJK->AbciJK', H.aa.vvvv[va, va, Va, Va], T.aaa.VVVoOO, optimize=True)
)
x3a += (1.0/4.0) * (
        -1.0*np.einsum('AmiE,EcbmJK->AbciJK', H.aa.voov[Va, oa, oa, Va], T.aaa.VvvoOO, optimize=True)
        -1.0*np.einsum('AMiE,EcbMJK->AbciJK', H.aa.voov[Va, Oa, oa, Va], T.aaa.VvvOOO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('bmie,AcemJK->AbciJK', H.aa.voov[va, oa, oa, va], T.aaa.VvvoOO, optimize=True)
        -1.0*np.einsum('bMie,AceMJK->AbciJK', H.aa.voov[va, Oa, oa, va], T.aaa.VvvOOO, optimize=True)
        +1.0*np.einsum('bmiE,AEcmJK->AbciJK', H.aa.voov[va, oa, oa, Va], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('bMiE,AEcMJK->AbciJK', H.aa.voov[va, Oa, oa, Va], T.aaa.VVvOOO, optimize=True)
)
x3a += (2.0/4.0) * (
        +1.0*np.einsum('AmJE,EcbmiK->AbciJK', H.aa.voov[Va, oa, Oa, Va], T.aaa.VvvooO, optimize=True)
        -1.0*np.einsum('AMJE,EcbiMK->AbciJK', H.aa.voov[Va, Oa, Oa, Va], T.aaa.VvvoOO, optimize=True)
)
x3a += (4.0/4.0) * (
        +1.0*np.einsum('bmJe,AcemiK->AbciJK', H.aa.voov[va, oa, Oa, va], T.aaa.VvvooO, optimize=True)
        -1.0*np.einsum('bMJe,AceiMK->AbciJK', H.aa.voov[va, Oa, Oa, va], T.aaa.VvvoOO, optimize=True)
        -1.0*np.einsum('bmJE,AEcmiK->AbciJK', H.aa.voov[va, oa, Oa, Va], T.aaa.VVvooO, optimize=True)
        +1.0*np.einsum('bMJE,AEciMK->AbciJK', H.aa.voov[va, Oa, Oa, Va], T.aaa.VVvoOO, optimize=True)
)
x3a += (1.0/4.0) * (
        -1.0*np.einsum('AmiE,cbEJKm->AbciJK', H.ab.voov[Va, ob, oa, Vb], T.aab.vvVOOo, optimize=True)
        -1.0*np.einsum('AMiE,cbEJKM->AbciJK', H.ab.voov[Va, Ob, oa, Vb], T.aab.vvVOOO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('bmie,AceJKm->AbciJK', H.ab.voov[va, ob, oa, vb], T.aab.VvvOOo, optimize=True)
        -1.0*np.einsum('bMie,AceJKM->AbciJK', H.ab.voov[va, Ob, oa, vb], T.aab.VvvOOO, optimize=True)
        -1.0*np.einsum('bmiE,AcEJKm->AbciJK', H.ab.voov[va, ob, oa, Vb], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('bMiE,AcEJKM->AbciJK', H.ab.voov[va, Ob, oa, Vb], T.aab.VvVOOO, optimize=True)
)
x3a += (2.0/4.0) * (
        +1.0*np.einsum('AmJE,cbEiKm->AbciJK', H.ab.voov[Va, ob, Oa, Vb], T.aab.vvVoOo, optimize=True)
        +1.0*np.einsum('AMJE,cbEiKM->AbciJK', H.ab.voov[Va, Ob, Oa, Vb], T.aab.vvVoOO, optimize=True)
)
x3a += (4.0/4.0) * (
        +1.0*np.einsum('bmJe,AceiKm->AbciJK', H.ab.voov[va, ob, Oa, vb], T.aab.VvvoOo, optimize=True)
        +1.0*np.einsum('bMJe,AceiKM->AbciJK', H.ab.voov[va, Ob, Oa, vb], T.aab.VvvoOO, optimize=True)
        +1.0*np.einsum('bmJE,AcEiKm->AbciJK', H.ab.voov[va, ob, Oa, Vb], T.aab.VvVoOo, optimize=True)
        +1.0*np.einsum('bMJE,AcEiKM->AbciJK', H.ab.voov[va, Ob, Oa, Vb], T.aab.VvVoOO, optimize=True)
)

x3a -= np.transpose(x3a, (0, 2, 1, 3, 4, 5))
x3a -= np.transpose(x3a, (0, 1, 2, 3, 5, 4))

get_error(X3A[Va, va, va, oa, Oa, Oa], x3a)


3.2028198917481508e-18


### Projection 9

In [19]:
x3a = (2.0/4.0) * (
        +1.0*np.einsum('mi,AcbmjK->AbcijK', H.a.oo[oa, oa], T.aaa.VvvooO, optimize=True)
        -1.0*np.einsum('Mi,AcbjMK->AbcijK', H.a.oo[Oa, oa], T.aaa.VvvoOO, optimize=True)
)
x3a += (1.0/4.0) * (
        -1.0*np.einsum('MK,AcbjiM->AbcijK', H.a.oo[Oa, Oa], T.aaa.VvvooO, optimize=True)
)
x3a += (1.0/4.0) * (
        -1.0*np.einsum('AE,EcbijK->AbcijK', H.a.vv[Va, Va], T.aaa.VvvooO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('be,AceijK->AbcijK', H.a.vv[va, va], T.aaa.VvvooO, optimize=True)
        +1.0*np.einsum('bE,AEcijK->AbcijK', H.a.vv[va, Va], T.aaa.VVvooO, optimize=True)
)
x3a += (1.0/4.0) * (
        -0.5*np.einsum('mnij,AcbmnK->AbcijK', H.aa.oooo[oa, oa, oa, oa], T.aaa.VvvooO, optimize=True)
        +1.0*np.einsum('Mnij,AcbnMK->AbcijK', H.aa.oooo[Oa, oa, oa, oa], T.aaa.VvvoOO, optimize=True)
        -0.5*np.einsum('MNij,AcbMNK->AbcijK', H.aa.oooo[Oa, Oa, oa, oa], T.aaa.VvvOOO, optimize=True)
)
x3a += (2.0/4.0) * (
        +1.0*np.einsum('MnKj,AcbniM->AbcijK', H.aa.oooo[Oa, oa, Oa, oa], T.aaa.VvvooO, optimize=True)
        +0.5*np.einsum('MNKj,AcbiMN->AbcijK', H.aa.oooo[Oa, Oa, Oa, oa], T.aaa.VvvoOO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('AbEf,EcfijK->AbcijK', H.aa.vvvv[Va, va, Va, va], T.aaa.VvvooO, optimize=True)
        -0.5*np.einsum('AbEF,FEcijK->AbcijK', H.aa.vvvv[Va, va, Va, Va], T.aaa.VVvooO, optimize=True)
)
x3a += (1.0/4.0) * (
        +0.5*np.einsum('cbef,AfeijK->AbcijK', H.aa.vvvv[va, va, va, va], T.aaa.VvvooO, optimize=True)
        +1.0*np.einsum('cbeF,AFeijK->AbcijK', H.aa.vvvv[va, va, va, Va], T.aaa.VVvooO, optimize=True)
        +0.5*np.einsum('cbEF,AFEijK->AbcijK', H.aa.vvvv[va, va, Va, Va], T.aaa.VVVooO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('AmiE,EcbmjK->AbcijK', H.aa.voov[Va, oa, oa, Va], T.aaa.VvvooO, optimize=True)
        +1.0*np.einsum('AMiE,EcbjMK->AbcijK', H.aa.voov[Va, Oa, oa, Va], T.aaa.VvvoOO, optimize=True)
)
x3a += (4.0/4.0) * (
        -1.0*np.einsum('bmie,AcemjK->AbcijK', H.aa.voov[va, oa, oa, va], T.aaa.VvvooO, optimize=True)
        +1.0*np.einsum('bMie,AcejMK->AbcijK', H.aa.voov[va, Oa, oa, va], T.aaa.VvvoOO, optimize=True)
        +1.0*np.einsum('bmiE,AEcmjK->AbcijK', H.aa.voov[va, oa, oa, Va], T.aaa.VVvooO, optimize=True)
        -1.0*np.einsum('bMiE,AEcjMK->AbcijK', H.aa.voov[va, Oa, oa, Va], T.aaa.VVvoOO, optimize=True)
)
x3a += (1.0/4.0) * (
        +1.0*np.einsum('AMKE,EcbjiM->AbcijK', H.aa.voov[Va, Oa, Oa, Va], T.aaa.VvvooO, optimize=True)
)
x3a += (2.0/4.0) * (
        +1.0*np.einsum('bMKe,AcejiM->AbcijK', H.aa.voov[va, Oa, Oa, va], T.aaa.VvvooO, optimize=True)
        -1.0*np.einsum('bMKE,AEcjiM->AbcijK', H.aa.voov[va, Oa, Oa, Va], T.aaa.VVvooO, optimize=True)
)
x3a += (2.0/4.0) * (
        -1.0*np.einsum('AmiE,cbEjKm->AbcijK', H.ab.voov[Va, ob, oa, Vb], T.aab.vvVoOo, optimize=True)
        -1.0*np.einsum('AMiE,cbEjKM->AbcijK', H.ab.voov[Va, Ob, oa, Vb], T.aab.vvVoOO, optimize=True)
)
x3a += (4.0/4.0) * (
        -1.0*np.einsum('bmie,AcejKm->AbcijK', H.ab.voov[va, ob, oa, vb], T.aab.VvvoOo, optimize=True)
        -1.0*np.einsum('bMie,AcejKM->AbcijK', H.ab.voov[va, Ob, oa, vb], T.aab.VvvoOO, optimize=True)
        -1.0*np.einsum('bmiE,AcEjKm->AbcijK', H.ab.voov[va, ob, oa, Vb], T.aab.VvVoOo, optimize=True)
        -1.0*np.einsum('bMiE,AcEjKM->AbcijK', H.ab.voov[va, Ob, oa, Vb], T.aab.VvVoOO, optimize=True)
)
x3a += (1.0/4.0) * (
        +1.0*np.einsum('AMKE,cbEjiM->AbcijK', H.ab.voov[Va, Ob, Oa, Vb], T.aab.vvVooO, optimize=True)
)
x3a += (2.0/4.0) * (
        +1.0*np.einsum('bMKe,AcejiM->AbcijK', H.ab.voov[va, Ob, Oa, vb], T.aab.VvvooO, optimize=True)
        +1.0*np.einsum('bMKE,AcEjiM->AbcijK', H.ab.voov[va, Ob, Oa, Vb], T.aab.VvVooO, optimize=True)
)

x3a -= np.transpose(x3a, (0, 2, 1, 3, 4, 5))
x3a -= np.transpose(x3a, (0, 1, 2, 4, 3, 5))

get_error(X3A[Va, va, va, oa, oa, Oa], x3a)


1.8510569572064956e-18


# Moments MM(2,3)

In [163]:
def build_intermediates(T, H):
    
    ##### I2A_vooo #####
    I2A_vooo = np.zeros_like(H.aa.vooo)
    # AmIJ
    I2A_vooo[Va, :, Oa, Oa] = (1.0/1.0) * (
        -0.5*np.einsum('mnef,AfenIJ->AmIJ', H.aa.oovv[:, oa, va, va], T.aaa.VvvoOO, optimize=True)
        -0.5*np.einsum('mNef,AfeIJN->AmIJ', H.aa.oovv[:, Oa, va, va], T.aaa.VvvOOO, optimize=True)
        +1.0*np.einsum('mneF,FAenIJ->AmIJ', H.aa.oovv[:, oa, va, Va], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('mNeF,FAeIJN->AmIJ', H.aa.oovv[:, Oa, va, Va], T.aaa.VVvOOO, optimize=True)
        -0.5*np.einsum('mnEF,FEAnIJ->AmIJ', H.aa.oovv[:, oa, Va, Va], T.aaa.VVVoOO, optimize=True)
        -0.5*np.einsum('mNEF,FEAIJN->AmIJ', H.aa.oovv[:, Oa, Va, Va], T.aaa.VVVOOO, optimize=True)
    )
    I2A_vooo[Va, :, Oa, Oa] += (1.0/1.0) * (
            +1.0*np.einsum('mnef,AefIJn->AmIJ', H.ab.oovv[:, ob, va, vb], T.aab.VvvOOo, optimize=True)
            +1.0*np.einsum('mNef,AefIJN->AmIJ', H.ab.oovv[:, Ob, va, vb], T.aab.VvvOOO, optimize=True)
            -1.0*np.einsum('mnEf,EAfIJn->AmIJ', H.ab.oovv[:, ob, Va, vb], T.aab.VVvOOo, optimize=True)
            -1.0*np.einsum('mNEf,EAfIJN->AmIJ', H.ab.oovv[:, Ob, Va, vb], T.aab.VVvOOO, optimize=True)
            +1.0*np.einsum('mneF,AeFIJn->AmIJ', H.ab.oovv[:, ob, va, Vb], T.aab.VvVOOo, optimize=True)
            +1.0*np.einsum('mNeF,AeFIJN->AmIJ', H.ab.oovv[:, Ob, va, Vb], T.aab.VvVOOO, optimize=True)
            -1.0*np.einsum('mnEF,EAFIJn->AmIJ', H.ab.oovv[:, ob, Va, Vb], T.aab.VVVOOo, optimize=True)
            -1.0*np.einsum('mNEF,EAFIJN->AmIJ', H.ab.oovv[:, Ob, Va, Vb], T.aab.VVVOOO, optimize=True)
    )
    I2A_vooo[Va, :, Oa, Oa] += H.aa.vooo[Va, :, Oa, Oa]
    # AmiJ
    I2A_vooo[Va, :, oa, Oa] = (1.0/1.0) * (
        +0.5*np.einsum('mnef,AfeinJ->AmiJ', H.aa.oovv[:, oa, va, va], T.aaa.VvvooO, optimize=True)
        -0.5*np.einsum('mNef,AfeiJN->AmiJ', H.aa.oovv[:, Oa, va, va], T.aaa.VvvoOO, optimize=True)
        +1.0*np.einsum('mnEf,EAfinJ->AmiJ', H.aa.oovv[:, oa, Va, va], T.aaa.VVvooO, optimize=True)
        -1.0*np.einsum('mNEf,EAfiJN->AmiJ', H.aa.oovv[:, Oa, Va, va], T.aaa.VVvoOO, optimize=True)
        +0.5*np.einsum('mnEF,FEAinJ->AmiJ', H.aa.oovv[:, oa, Va, Va], T.aaa.VVVooO, optimize=True)
        -0.5*np.einsum('mNEF,FEAiJN->AmiJ', H.aa.oovv[:, Oa, Va, Va], T.aaa.VVVoOO, optimize=True)
    )
    I2A_vooo[Va, :, oa, Oa] += (1.0/1.0) * (
            +1.0*np.einsum('mnef,AefiJn->AmiJ', H.ab.oovv[:, ob, va, vb], T.aab.VvvoOo, optimize=True)
            +1.0*np.einsum('mNef,AefiJN->AmiJ', H.ab.oovv[:, Ob, va, vb], T.aab.VvvoOO, optimize=True)
            +1.0*np.einsum('mneF,AeFiJn->AmiJ', H.ab.oovv[:, ob, va, Vb], T.aab.VvVoOo, optimize=True)
            +1.0*np.einsum('mNeF,AeFiJN->AmiJ', H.ab.oovv[:, Ob, va, Vb], T.aab.VvVoOO, optimize=True)
            -1.0*np.einsum('mnEf,EAfiJn->AmiJ', H.ab.oovv[:, ob, Va, vb], T.aab.VVvoOo, optimize=True)
            -1.0*np.einsum('mNEf,EAfiJN->AmiJ', H.ab.oovv[:, Ob, Va, vb], T.aab.VVvoOO, optimize=True)
            -1.0*np.einsum('mnEF,EAFiJn->AmiJ', H.ab.oovv[:, ob, Va, Vb], T.aab.VVVoOo, optimize=True)
            -1.0*np.einsum('mNEF,EAFiJN->AmiJ', H.ab.oovv[:, Ob, Va, Vb], T.aab.VVVoOO, optimize=True)
    )
    I2A_vooo[Va, :, oa, Oa] += H.aa.vooo[Va, :, oa, Oa]
    I2A_vooo[Va, :, Oa, oa] = -1.0*np.transpose(I2A_vooo[Va, :, oa, Oa], (0, 1, 3, 2))
    # Amij
    I2A_vooo[Va, :, oa, oa] = (1.0/1.0) * (
        -0.5*np.einsum('mNef,AfeijN->Amij', H.aa.oovv[:, Oa, va, va], T.aaa.VvvooO, optimize=True)
        -1.0*np.einsum('mNEf,EAfijN->Amij', H.aa.oovv[:, Oa, Va, va], T.aaa.VVvooO, optimize=True)
        -0.5*np.einsum('mNEF,FEAijN->Amij', H.aa.oovv[:, Oa, Va, Va], T.aaa.VVVooO, optimize=True)
    )
    I2A_vooo[Va, :, oa, oa] += (1.0/1.0) * (
            +1.0*np.einsum('mNef,AefijN->Amij', H.ab.oovv[:, Ob, va, vb], T.aab.VvvooO, optimize=True)
            +1.0*np.einsum('mNeF,AeFijN->Amij', H.ab.oovv[:, Ob, va, Vb], T.aab.VvVooO, optimize=True)
            -1.0*np.einsum('mNEf,EAfijN->Amij', H.ab.oovv[:, Ob, Va, vb], T.aab.VVvooO, optimize=True)
            -1.0*np.einsum('mNEF,EAFijN->Amij', H.ab.oovv[:, Ob, Va, Vb], T.aab.VVVooO, optimize=True)
    )
    I2A_vooo[Va, :, oa, oa] += H.aa.vooo[Va, :, oa, oa]
    # amIJ
    I2A_vooo[va, :, Oa, Oa] = (1.0/1.0) * (
        +1.0*np.einsum('mnEf,EfanIJ->amIJ', H.aa.oovv[:, oa, Va, va], T.aaa.VvvoOO, optimize=True)
        -0.5*np.einsum('mnEF,FEanIJ->amIJ', H.aa.oovv[:, oa, Va, Va], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('mNEf,EfaIJN->amIJ', H.aa.oovv[:, Oa, Va, va], T.aaa.VvvOOO, optimize=True)
        -0.5*np.einsum('mNEF,FEaIJN->amIJ', H.aa.oovv[:, Oa, Va, Va], T.aaa.VVvOOO, optimize=True)
    )
    I2A_vooo[va, :, Oa, Oa] += (1.0/1.0) * (
            -1.0*np.einsum('mneF,eaFIJn->amIJ', H.ab.oovv[:, ob, va, Vb], T.aab.vvVOOo, optimize=True)
            -1.0*np.einsum('mnEf,EafIJn->amIJ', H.ab.oovv[:, ob, Va, vb], T.aab.VvvOOo, optimize=True)
            -1.0*np.einsum('mnEF,EaFIJn->amIJ', H.ab.oovv[:, ob, Va, Vb], T.aab.VvVOOo, optimize=True)
            -1.0*np.einsum('mNeF,eaFIJN->amIJ', H.ab.oovv[:, Ob, va, Vb], T.aab.vvVOOO, optimize=True)
            -1.0*np.einsum('mNEf,EafIJN->amIJ', H.ab.oovv[:, Ob, Va, vb], T.aab.VvvOOO, optimize=True)
            -1.0*np.einsum('mNEF,EaFIJN->amIJ', H.ab.oovv[:, Ob, Va, Vb], T.aab.VvVOOO, optimize=True)
    )
    I2A_vooo[va, :, Oa, Oa] += H.aa.vooo[va, :, Oa, Oa]
    # amIj
    I2A_vooo[va, :, Oa, oa] = (1.0/1.0) * (
        -1.0*np.einsum('mneF,FeajnI->amIj', H.aa.oovv[:, oa, va, Va], T.aaa.VvvooO, optimize=True)
        -0.5*np.einsum('mnEF,FEajnI->amIj', H.aa.oovv[:, oa, Va, Va], T.aaa.VVvooO, optimize=True)
        +1.0*np.einsum('mNeF,FeajIN->amIj', H.aa.oovv[:, Oa, va, Va], T.aaa.VvvoOO, optimize=True)
        +0.5*np.einsum('mNEF,FEajIN->amIj', H.aa.oovv[:, Oa, Va, Va], T.aaa.VVvoOO, optimize=True)
    )
    I2A_vooo[va, :, Oa, oa] += (1.0/1.0) * (
            +1.0*np.einsum('mnEf,EafjIn->amIj', H.ab.oovv[:, ob, Va, vb], T.aab.VvvoOo, optimize=True)
            +1.0*np.einsum('mneF,eaFjIn->amIj', H.ab.oovv[:, ob, va, Vb], T.aab.vvVoOo, optimize=True)
            +1.0*np.einsum('mnEF,EaFjIn->amIj', H.ab.oovv[:, ob, Va, Vb], T.aab.VvVoOo, optimize=True)
            +1.0*np.einsum('mNEf,EafjIN->amIj', H.ab.oovv[:, Ob, Va, vb], T.aab.VvvoOO, optimize=True)
            +1.0*np.einsum('mNeF,eaFjIN->amIj', H.ab.oovv[:, Ob, va, Vb], T.aab.vvVoOO, optimize=True)
            +1.0*np.einsum('mNEF,EaFjIN->amIj', H.ab.oovv[:, Ob, Va, Vb], T.aab.VvVoOO, optimize=True)
    )
    I2A_vooo[va, :, Oa, oa] += H.aa.vooo[va, :, Oa, oa]
    I2A_vooo[va, :, oa, Oa] = -1.0*np.transpose(I2A_vooo[va, :, Oa, oa], (0, 1, 3, 2))
    # amij
    I2A_vooo[va, :, oa, oa] = (1.0/1.0) * (
        +1.0*np.einsum('mNEf,EfaijN->amij', H.aa.oovv[:, Oa, Va, va], T.aaa.VvvooO, optimize=True)
        -0.5*np.einsum('mNEF,FEaijN->amij', H.aa.oovv[:, Oa, Va, Va], T.aaa.VVvooO, optimize=True)
    )
    I2A_vooo[va, :, oa, oa] += (1.0/1.0) * (
            -1.0*np.einsum('mNeF,eaFijN->amij', H.ab.oovv[:, Ob, va, Vb], T.aab.vvVooO, optimize=True)
            -1.0*np.einsum('mNEf,EafijN->amij', H.ab.oovv[:, Ob, Va, vb], T.aab.VvvooO, optimize=True)
            -1.0*np.einsum('mNEF,EaFijN->amij', H.ab.oovv[:, Ob, Va, Vb], T.aab.VvVooO, optimize=True)
    )
    I2A_vooo[va, :, oa, oa] += H.aa.vooo[va, :, oa, oa]
    
    
    
    ##### I2A_vvov #####
    I2A_vvov = H.aa.vvov + np.einsum("me,abim->abie", H.a.ov, T0.aa, optimize=True)
    # ABIe
    I2A_vvov[Va, Va, Oa, :] += (1.0/1.0) * (
        +0.5*np.einsum('mnef,BAfmnI->ABIe', H.aa.oovv[oa, oa, :, va], T.aaa.VVvooO, optimize=True)
        +0.5*np.einsum('mneF,FBAmnI->ABIe', H.aa.oovv[oa, oa, :, Va], T.aaa.VVVooO, optimize=True)
        -1.0*np.einsum('mNef,BAfmIN->ABIe', H.aa.oovv[oa, Oa, :, va], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('mNeF,FBAmIN->ABIe', H.aa.oovv[oa, Oa, :, Va], T.aaa.VVVoOO, optimize=True)
        +0.5*np.einsum('MNef,BAfIMN->ABIe', H.aa.oovv[Oa, Oa, :, va], T.aaa.VVvOOO, optimize=True)
        +0.5*np.einsum('MNeF,FBAIMN->ABIe', H.aa.oovv[Oa, Oa, :, Va], T.aaa.VVVOOO, optimize=True)
    )
    I2A_vvov[Va, Va, Oa, :] += (1.0/1.0) * (
            -1.0*np.einsum('mnef,BAfmIn->ABIe', H.ab.oovv[oa, ob, :, vb], T.aab.VVvoOo, optimize=True)
            -1.0*np.einsum('mneF,BAFmIn->ABIe', H.ab.oovv[oa, ob, :, Vb], T.aab.VVVoOo, optimize=True)
            +1.0*np.einsum('Mnef,BAfIMn->ABIe', H.ab.oovv[Oa, ob, :, vb], T.aab.VVvOOo, optimize=True)
            +1.0*np.einsum('MneF,BAFIMn->ABIe', H.ab.oovv[Oa, ob, :, Vb], T.aab.VVVOOo, optimize=True)
            -1.0*np.einsum('mNef,BAfmIN->ABIe', H.ab.oovv[oa, Ob, :, vb], T.aab.VVvoOO, optimize=True)
            -1.0*np.einsum('mNeF,BAFmIN->ABIe', H.ab.oovv[oa, Ob, :, Vb], T.aab.VVVoOO, optimize=True)
            +1.0*np.einsum('MNef,BAfIMN->ABIe', H.ab.oovv[Oa, Ob, :, vb], T.aab.VVvOOO, optimize=True)
            +1.0*np.einsum('MNeF,BAFIMN->ABIe', H.ab.oovv[Oa, Ob, :, Vb], T.aab.VVVOOO, optimize=True)
    )
    # aBIe
    I2A_vvov[va, Va, Oa, :] += (1.0/1.0) * (
        -0.5*np.einsum('mnef,BfamnI->aBIe', H.aa.oovv[oa, oa, :, va], T.aaa.VvvooO, optimize=True)
        +0.5*np.einsum('mneF,FBamnI->aBIe', H.aa.oovv[oa, oa, :, Va], T.aaa.VVvooO, optimize=True)
        +1.0*np.einsum('mNef,BfamIN->aBIe', H.aa.oovv[oa, Oa, :, va], T.aaa.VvvoOO, optimize=True)
        -1.0*np.einsum('mNeF,FBamIN->aBIe', H.aa.oovv[oa, Oa, :, Va], T.aaa.VVvoOO, optimize=True)
        -0.5*np.einsum('MNef,BfaIMN->aBIe', H.aa.oovv[Oa, Oa, :, va], T.aaa.VvvOOO, optimize=True)
        +0.5*np.einsum('MNeF,FBaIMN->aBIe', H.aa.oovv[Oa, Oa, :, Va], T.aaa.VVvOOO, optimize=True)
    )
    I2A_vvov[va, Va, Oa, :] += (1.0/1.0) * (
            -1.0*np.einsum('mnef,BafmIn->aBIe', H.ab.oovv[oa, ob, :, vb], T.aab.VvvoOo, optimize=True)
            -1.0*np.einsum('mneF,BaFmIn->aBIe', H.ab.oovv[oa, ob, :, Vb], T.aab.VvVoOo, optimize=True)
            +1.0*np.einsum('Mnef,BafIMn->aBIe', H.ab.oovv[Oa, ob, :, vb], T.aab.VvvOOo, optimize=True)
            +1.0*np.einsum('MneF,BaFIMn->aBIe', H.ab.oovv[Oa, ob, :, Vb], T.aab.VvVOOo, optimize=True)
            -1.0*np.einsum('mNef,BafmIN->aBIe', H.ab.oovv[oa, Ob, :, vb], T.aab.VvvoOO, optimize=True)
            -1.0*np.einsum('mNeF,BaFmIN->aBIe', H.ab.oovv[oa, Ob, :, Vb], T.aab.VvVoOO, optimize=True)
            +1.0*np.einsum('MNef,BafIMN->aBIe', H.ab.oovv[Oa, Ob, :, vb], T.aab.VvvOOO, optimize=True)
            +1.0*np.einsum('MNeF,BaFIMN->aBIe', H.ab.oovv[Oa, Ob, :, Vb], T.aab.VvVOOO, optimize=True)
    )
    I2A_vvov[Va, va, Oa, :] = -1.0*np.transpose(I2A_vvov[va, Va, Oa, :], (1, 0, 2, 3))
    # abIe
    I2A_vvov[va, va, Oa, :] += (1.0/1.0) * (
        +0.5*np.einsum('mneF,FbamnI->abIe', H.aa.oovv[oa, oa, :, Va], T.aaa.VvvooO, optimize=True)
        -1.0*np.einsum('mNeF,FbamIN->abIe', H.aa.oovv[oa, Oa, :, Va], T.aaa.VvvoOO, optimize=True)
        +0.5*np.einsum('MNeF,FbaIMN->abIe', H.aa.oovv[Oa, Oa, :, Va], T.aaa.VvvOOO, optimize=True)
    )
    I2A_vvov[va, va, Oa, :] += (1.0/1.0) * (
            -1.0*np.einsum('mneF,baFmIn->abIe', H.ab.oovv[oa, ob, :, Vb], T.aab.vvVoOo, optimize=True)
            +1.0*np.einsum('MneF,baFIMn->abIe', H.ab.oovv[Oa, ob, :, Vb], T.aab.vvVOOo, optimize=True)
            -1.0*np.einsum('mNeF,baFmIN->abIe', H.ab.oovv[oa, Ob, :, Vb], T.aab.vvVoOO, optimize=True)
            +1.0*np.einsum('MNeF,baFIMN->abIe', H.ab.oovv[Oa, Ob, :, Vb], T.aab.vvVOOO, optimize=True)
    )
    # ABie
    I2A_vvov[Va, Va, oa, :] += (1.0/1.0) * (
        +1.0*np.einsum('mNef,BAfimN->ABie', H.aa.oovv[oa, Oa, :, va], T.aaa.VVvooO, optimize=True)
        +1.0*np.einsum('mNeF,FBAimN->ABie', H.aa.oovv[oa, Oa, :, Va], T.aaa.VVVooO, optimize=True)
        +0.5*np.einsum('MNef,BAfiMN->ABie', H.aa.oovv[Oa, Oa, :, va], T.aaa.VVvoOO, optimize=True)
        +0.5*np.einsum('MNeF,FBAiMN->ABie', H.aa.oovv[Oa, Oa, :, Va], T.aaa.VVVoOO, optimize=True)
    )
    I2A_vvov[Va, Va, oa, :] += (1.0/1.0) * (
            +1.0*np.einsum('Mnef,BAfiMn->ABie', H.ab.oovv[Oa, ob, :, vb], T.aab.VVvoOo, optimize=True)
            +1.0*np.einsum('MneF,BAFiMn->ABie', H.ab.oovv[Oa, ob, :, Vb], T.aab.VVVoOo, optimize=True)
            +1.0*np.einsum('mNef,BAfimN->ABie', H.ab.oovv[oa, Ob, :, vb], T.aab.VVvooO, optimize=True)
            +1.0*np.einsum('mNeF,BAFimN->ABie', H.ab.oovv[oa, Ob, :, Vb], T.aab.VVVooO, optimize=True)
            +1.0*np.einsum('MNef,BAfiMN->ABie', H.ab.oovv[Oa, Ob, :, vb], T.aab.VVvoOO, optimize=True)
            +1.0*np.einsum('MNeF,BAFiMN->ABie', H.ab.oovv[Oa, Ob, :, Vb], T.aab.VVVoOO, optimize=True)
    )
    # Abie
    I2A_vvov[Va, va, oa, :] += (1.0/1.0) * (
        +1.0*np.einsum('mNef,AfbimN->Abie', H.aa.oovv[oa, Oa, :, va], T.aaa.VvvooO, optimize=True)
        +0.5*np.einsum('MNef,AfbiMN->Abie', H.aa.oovv[Oa, Oa, :, va], T.aaa.VvvoOO, optimize=True)
        -1.0*np.einsum('mNeF,FAbimN->Abie', H.aa.oovv[oa, Oa, :, Va], T.aaa.VVvooO, optimize=True)
        -0.5*np.einsum('MNeF,FAbiMN->Abie', H.aa.oovv[Oa, Oa, :, Va], T.aaa.VVvoOO, optimize=True)
    )
    I2A_vvov[Va, va, oa, :] += (1.0/1.0) * (
            -1.0*np.einsum('Mnef,AbfiMn->Abie', H.ab.oovv[Oa, ob, :, vb], T.aab.VvvoOo, optimize=True)
            -1.0*np.einsum('mNef,AbfimN->Abie', H.ab.oovv[oa, Ob, :, vb], T.aab.VvvooO, optimize=True)
            -1.0*np.einsum('MNef,AbfiMN->Abie', H.ab.oovv[Oa, Ob, :, vb], T.aab.VvvoOO, optimize=True)
            -1.0*np.einsum('MneF,AbFiMn->Abie', H.ab.oovv[Oa, ob, :, Vb], T.aab.VvVoOo, optimize=True)
            -1.0*np.einsum('mNeF,AbFimN->Abie', H.ab.oovv[oa, Ob, :, Vb], T.aab.VvVooO, optimize=True)
            -1.0*np.einsum('MNeF,AbFiMN->Abie', H.ab.oovv[Oa, Ob, :, Vb], T.aab.VvVoOO, optimize=True)
    )
    I2A_vvov[va, Va, oa, :] = -1.0*np.transpose(I2A_vvov[Va, va, oa, :], (1, 0, 2, 3))
    # abie
    I2A_vvov[va, va, oa, :] += (1.0/1.0) * (
            -1.0*np.einsum('MneF,FbainM->abie', H.aa.oovv[Oa, oa, :, Va], T.aaa.VvvooO, optimize=True)
            +0.5*np.einsum('MNeF,FbaiMN->abie', H.aa.oovv[Oa, Oa, :, Va], T.aaa.VvvoOO, optimize=True)
    )
    I2A_vvov[va, va, oa, :] += (1.0/1.0) * (
            +1.0*np.einsum('mNeF,baFimN->abie', H.ab.oovv[oa, Ob, :, Vb], T.aab.vvVooO, optimize=True)
            +1.0*np.einsum('MneF,baFiMn->abie', H.ab.oovv[Oa, ob, :, Vb], T.aab.vvVoOo, optimize=True)
            +1.0*np.einsum('MNeF,baFiMN->abie', H.ab.oovv[Oa, Ob, :, Vb], T.aab.vvVoOO, optimize=True)
    )

    
    return I2A_vooo, I2A_vvov

In [164]:
I2A_vooo = 0.5 * np.einsum("mnef,aefijn->amij", H.aa.oovv, T0.aaa, optimize=True)
I2A_vooo += H.aa.vooo + np.einsum("mnef,aefijn->amij", H.ab.oovv, T0.aab, optimize=True)

I2A_vvov = -0.5 * np.einsum("mnef,abfimn->abie", H.aa.oovv, T0.aaa, optimize=True)
I2A_vvov -= np.einsum("mnef,abfimn->abie", H.ab.oovv, T0.aab, optimize=True)
I2A_vvov += H.aa.vvov + np.einsum("me,abim->abie", H.a.ov, T0.aa, optimize=True)

I2A_vooo_2, I2A_vvov_2 = build_intermediates(T, H)
get_error(I2A_vooo, I2A_vooo_2)
get_error(I2A_vvov, I2A_vvov_2)

9.089293858866171e-17
9.383862073614989e-15


In [35]:
I2A_vvov = H.aa.vvov + np.einsum("me,abim->abie", H.a.ov, T0.aa, optimize=True)

D1 = -0.25 * np.einsum('amij,bcmk->abcijk', H.aa.vooo, T0.aa, optimize=True)
D2 = 0.25 * np.einsum('abie,ecjk->abcijk', I2A_vvov, T0.aa, optimize=True)

D1 -= np.transpose(D1, (1, 0, 2, 3, 4, 5)) + np.transpose(D1, (2, 1, 0, 3, 4, 5))
D1 -= np.transpose(D1, (0, 1, 2, 5, 4, 3)) + np.transpose(D1, (0, 1, 2, 3, 5, 4))

In [36]:
d1 = -(1.0/4.0) * np.einsum('Amij,bcmK->AbcijK', H.aa.vooo[Va, :, :, :], T0.aa[:, :, :, Oa], optimize=True)
d1 -= (4.0/4.0) * np.einsum('cmKj,Abim->AbcijK', H.aa.vooo[:, :, Oa, :], T0.aa[Va, :, :, :], optimize=True)
d1 -= (2.0/4.0) * np.einsum('bmji,AcmK->AbcijK', H.aa.vooo, T0.aa[Va, :, :, Oa], optimize=True)
d1 -= (2.0/4.0) * np.einsum('AmiK,bcjm->AbcijK', H.aa.vooo[Va, :, :, Oa], T0.aa, optimize=True)

d1 -= np.transpose(d1, (0, 2, 1, 3, 4, 5))
d1 -= np.transpose(d1, (0, 1, 2, 4, 3, 5))

In [38]:
get_error(D1[Va, Va, Va, Oa, Oa, Oa], d1[Va, Va, Va, Oa, Oa, Oa])

0.10769848989544732


In [126]:
I2A_vvov = -0.5 * np.einsum("mnef,abfimn->abie", H.aa.oovv, T0.aaa, optimize=True)
I2A_vvov -= np.einsum("mnef,abfimn->abie", H.ab.oovv, T0.aab, optimize=True)
I2A_vvov += H.aa.vvov + np.einsum("me,abim->abie", H.a.ov, T0.aa, optimize=True)

I2A_vooo = 0.5 * np.einsum("mnef,aefijn->amij", H.aa.oovv, T0.aaa, optimize=True)
I2A_vooo += H.aa.vooo + np.einsum("mnef,aefijn->amij", H.ab.oovv, T0.aab, optimize=True)

# MM(2,3)A
MM23_D1 = -np.einsum("amij,bcmk->abcijk", I2A_vooo, T0.aa, optimize=True)
MM23_D1 -= np.transpose(MM23_D1, (0, 1, 2, 5, 4, 3)) + np.transpose(MM23_D1, (0, 1, 2, 3, 5, 4))
MM23_D1 -= np.transpose(MM23_D1, (1, 0, 2, 3, 4, 5)) + np.transpose(MM23_D1, (2, 1, 0, 3, 4, 5))

MM23_D2 = np.einsum("abie,ecjk->abcijk", I2A_vvov, T0.aa, optimize=True)
MM23_D2 -= np.transpose(MM23_D2, (0, 1, 2, 4, 3, 5)) + np.transpose(MM23_D2, (0, 1, 2, 5, 4, 3))
MM23_D2 -= np.transpose(MM23_D2, (2, 1, 0, 3, 4, 5)) + np.transpose(MM23_D2, (0, 2, 1, 3, 4, 5))

D1 = -np.einsum("mk,abcijm->abcijk", H.a.oo, T0.aaa, optimize=True)
D1 -= np.transpose(D1, (0, 1, 2, 5, 4, 3)) + np.transpose(D1, (0, 1, 2, 3, 5, 4))

D2 = np.einsum('ae,ebcijk->abcijk', H.a.vv, T0.aaa, optimize=True)
D2 -= np.transpose(D2, (1, 0, 2, 3, 4, 5)) + np.transpose(D2, (2, 1, 0, 3, 4, 5))

D3 = 0.5 * np.einsum('mnij,abcmnk->abcijk', H.aa.oooo, T0.aaa, optimize=True)
D3 -= np.transpose(D3, (0, 1, 2, 5, 4, 3)) + np.transpose(D3, (0, 1, 2, 3, 5, 4))

D4 = 0.5 * np.einsum('abef,efcijk->abcijk', H.aa.vvvv, T0.aaa, optimize=True)
D4 -= np.transpose(D4, (2, 1, 0, 3, 4, 5)) + np.transpose(D4, (0, 2, 1, 3, 4, 5))

D5 = np.einsum('amie,ebcmjk->abcijk', H.aa.voov, T0.aaa, optimize=True)
D5 -= np.transpose(D5, (1, 0, 2, 3, 4, 5)) + np.transpose(D5, (2, 1, 0, 3, 4, 5))
D5 -= np.transpose(D5, (0, 1, 2, 4, 3, 5)) + np.transpose(D5, (0, 1, 2, 5, 4, 3))

D6 = np.einsum('amie,bcejkm->abcijk', H.ab.voov, T0.aab, optimize=True)
D6 -= np.transpose(D6, (1, 0, 2, 3, 4, 5)) + np.transpose(D6, (2, 1, 0, 3, 4, 5))
D6 -= np.transpose(D6, (0, 1, 2, 4, 3, 5)) + np.transpose(D6, (0, 1, 2, 5, 4, 3))


X3A = D1 + D2 + D3 + D4 + D5 + D6 + MM23_D1# + MM23_D2

In [129]:
def update(T, H, I2A_vvov, H0, shift, system):
    
    oa = slice(0, noccupied_alpha - nact_occupied_alpha)
    Oa = slice(noccupied_alpha - nact_occupied_alpha, noccupied_alpha)
    va = slice(nact_unoccupied_alpha, nunoccupied_alpha)
    Va = slice(0, nact_unoccupied_alpha)

    ob = slice(0, noccupied_beta - nact_occupied_beta)
    Ob = slice(noccupied_beta - nact_occupied_beta, noccupied_beta)
    vb = slice(nact_unoccupied_beta, nunoccupied_beta)
    Vb = slice(0, nact_unoccupied_beta)


    
    # MM(2,3)A
    x3a = -(6.0/12.0) * np.einsum("AmiJ,BCmK->ABCiJK", I2A_VooO, T.aa[Va, Va, :, Oa], optimize=True)
    #x3a -= -(3.0/12.0) * np.einsum("AmKJ,BCmi->ABCiJK", I2A_VoOO, T.aa[Va, Va, :, oa], optimize=True)
    
    x3a += (3.0/12.0) * np.einsum("ABie,eCJK->ABCiJK", I2A_VVov, T.aa[:, Va, Oa, Oa], optimize=True)
    x3a -= (6.0/12.0) * np.einsum("ABJe,eCiK->ABCiJK", I2A_VVOv, T.aa[:, Va, oa, Oa], optimize=True)

    x3a += (1.0 / 12.0) * (
            +1.0 * np.einsum('mi,CBAmJK->ABCiJK', H.a.oo[oa, oa], T.aaa.VVVoOO, optimize=True)
            + 1.0 * np.einsum('Mi,CBAMJK->ABCiJK', H.a.oo[Oa, oa], T.aaa.VVVOOO, optimize=True)
    )
    x3a += (2.0 / 12.0) * (
            -1.0 * np.einsum('mJ,CBAmiK->ABCiJK', H.a.oo[oa, Oa], T.aaa.VVVooO, optimize=True)
            + 1.0 * np.einsum('MJ,CBAiMK->ABCiJK', H.a.oo[Oa, Oa], T.aaa.VVVoOO, optimize=True)
    )
    x3a += (3.0 / 12.0) * (
            -1.0 * np.einsum('Ae,CBeiJK->ABCiJK', H.a.vv[Va, va], T.aaa.VVvoOO, optimize=True)
            - 1.0 * np.einsum('AE,CBEiJK->ABCiJK', H.a.vv[Va, Va], T.aaa.VVVoOO, optimize=True)
    )
    x3a += (2.0 / 12.0) * (
            -0.5 * np.einsum('mniJ,CBAmnK->ABCiJK', H.aa.oooo[oa, oa, oa, Oa], T.aaa.VVVooO, optimize=True)
            + 1.0 * np.einsum('MniJ,CBAnMK->ABCiJK', H.aa.oooo[Oa, oa, oa, Oa], T.aaa.VVVoOO, optimize=True)
            - 0.5 * np.einsum('MNiJ,CBAMNK->ABCiJK', H.aa.oooo[Oa, Oa, oa, Oa], T.aaa.VVVOOO, optimize=True)
    )
    x3a += (1.0 / 12.0) * (
            +1.0 * np.einsum('MnKJ,CBAniM->ABCiJK', H.aa.oooo[Oa, oa, Oa, Oa], T.aaa.VVVooO, optimize=True)
            + 0.5 * np.einsum('MNKJ,CBAiMN->ABCiJK', H.aa.oooo[Oa, Oa, Oa, Oa], T.aaa.VVVoOO, optimize=True)
    )
    x3a += (3.0 / 12.0) * (
            -0.5 * np.einsum('ABef,CfeiJK->ABCiJK', H.aa.vvvv[Va, Va, va, va], T.aaa.VvvoOO, optimize=True)
            + 1.0 * np.einsum('ABEf,CEfiJK->ABCiJK', H.aa.vvvv[Va, Va, Va, va], T.aaa.VVvoOO, optimize=True)
            - 0.5 * np.einsum('ABEF,CFEiJK->ABCiJK', H.aa.vvvv[Va, Va, Va, Va], T.aaa.VVVoOO, optimize=True)
    )
    x3a += (3.0 / 12.0) * (
            -1.0 * np.einsum('Amie,CBemJK->ABCiJK', H.aa.voov[Va, oa, oa, va], T.aaa.VVvoOO, optimize=True)
            - 1.0 * np.einsum('AmiE,CBEmJK->ABCiJK', H.aa.voov[Va, oa, oa, Va], T.aaa.VVVoOO, optimize=True)
            - 1.0 * np.einsum('AMie,CBeMJK->ABCiJK', H.aa.voov[Va, Oa, oa, va], T.aaa.VVvOOO, optimize=True)
            - 1.0 * np.einsum('AMiE,CBEMJK->ABCiJK', H.aa.voov[Va, Oa, oa, Va], T.aaa.VVVOOO, optimize=True)
    )
    x3a += (6.0 / 12.0) * (
            +1.0 * np.einsum('AmJe,CBemiK->ABCiJK', H.aa.voov[Va, oa, Oa, va], T.aaa.VVvooO, optimize=True)
            + 1.0 * np.einsum('AmJE,CBEmiK->ABCiJK', H.aa.voov[Va, oa, Oa, Va], T.aaa.VVVooO, optimize=True)
            - 1.0 * np.einsum('AMJe,CBeiMK->ABCiJK', H.aa.voov[Va, Oa, Oa, va], T.aaa.VVvoOO, optimize=True)
            - 1.0 * np.einsum('AMJE,CBEiMK->ABCiJK', H.aa.voov[Va, Oa, Oa, Va], T.aaa.VVVoOO, optimize=True)
    )
    x3a += (3.0 / 12.0) * (
            -1.0 * np.einsum('Amie,CBeJKm->ABCiJK', H.ab.voov[Va, ob, oa, vb], T.aab.VVvOOo, optimize=True)
            - 1.0 * np.einsum('AmiE,CBEJKm->ABCiJK', H.ab.voov[Va, ob, oa, Vb], T.aab.VVVOOo, optimize=True)
            - 1.0 * np.einsum('AMie,CBeJKM->ABCiJK', H.ab.voov[Va, Ob, oa, vb], T.aab.VVvOOO, optimize=True)
            - 1.0 * np.einsum('AMiE,CBEJKM->ABCiJK', H.ab.voov[Va, Ob, oa, Vb], T.aab.VVVOOO, optimize=True)
    )
    x3a += (6.0 / 12.0) * (
            +1.0 * np.einsum('AmJe,CBeiKm->ABCiJK', H.ab.voov[Va, ob, Oa, vb], T.aab.VVvoOo, optimize=True)
            + 1.0 * np.einsum('AmJE,CBEiKm->ABCiJK', H.ab.voov[Va, ob, Oa, Vb], T.aab.VVVoOo, optimize=True)
            + 1.0 * np.einsum('AMJe,CBeiKM->ABCiJK', H.ab.voov[Va, Ob, Oa, vb], T.aab.VVvoOO, optimize=True)
            + 1.0 * np.einsum('AMJE,CBEiKM->ABCiJK', H.ab.voov[Va, Ob, Oa, Vb], T.aab.VVVoOO, optimize=True)
    )

    x3a -= np.transpose(x3a, (1, 0, 2, 3, 4, 5)) + np.transpose(x3a, (0, 2, 1, 3, 4, 5)) \
           + np.transpose(x3a, (2, 1, 0, 3, 4, 5)) - np.transpose(x3a, (1, 2, 0, 3, 4, 5)) \
           - np.transpose(x3a, (2, 0, 1, 3, 4, 5))

    x3a -= np.transpose(x3a, (0, 1, 2, 3, 5, 4))

    return x3a

In [130]:
I2A_vvov = H.aa.vvov + np.einsum("me,abim->abie", H.a.ov, T0.aa, optimize=True)
x3a = update(T, H, I2A_vvov, H0, shift, system)

In [131]:
get_error(X3A[Va, Va, Va, oa, Oa, Oa], x3a)

0.09671839372786285


In [144]:
I2A_vvov_full = -0.5 * np.einsum("mnef,abfimn->abie", H.aa.oovv, T0.aaa, optimize=True)
I2A_vvov_full -= np.einsum("mnef,abfimn->abie", H.ab.oovv, T0.aab, optimize=True)
I2A_vvov_full += H.aa.vvov + np.einsum("me,abim->abie", H.a.ov, T0.aa, optimize=True)

I2A_vooo_full = 0.5 * np.einsum("mnef,aefijn->amij", H.aa.oovv, T0.aaa, optimize=True)
I2A_vooo_full += H.aa.vooo + np.einsum("mnef,aefijn->amij", H.ab.oovv, T0.aab, optimize=True)

In [145]:
I2A_VoOO = (1.0/1.0) * (
        -0.5*np.einsum('mnef,AfenIJ->AmIJ', H.aa.oovv[:, oa, va, va], T.aaa.VvvoOO, optimize=True)
        -0.5*np.einsum('mNef,AfeIJN->AmIJ', H.aa.oovv[:, Oa, va, va], T.aaa.VvvOOO, optimize=True)
        +1.0*np.einsum('mneF,FAenIJ->AmIJ', H.aa.oovv[:, oa, va, Va], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('mNeF,FAeIJN->AmIJ', H.aa.oovv[:, Oa, va, Va], T.aaa.VVvOOO, optimize=True)
        -0.5*np.einsum('mnEF,FEAnIJ->AmIJ', H.aa.oovv[:, oa, Va, Va], T.aaa.VVVoOO, optimize=True)
        -0.5*np.einsum('mNEF,FEAIJN->AmIJ', H.aa.oovv[:, Oa, Va, Va], T.aaa.VVVOOO, optimize=True)
)
I2A_VoOO += (1.0/1.0) * (
        +1.0*np.einsum('mnef,AefIJn->AmIJ', H.ab.oovv[:, ob, va, vb], T.aab.VvvOOo, optimize=True)
        +1.0*np.einsum('mNef,AefIJN->AmIJ', H.ab.oovv[:, Ob, va, vb], T.aab.VvvOOO, optimize=True)
        -1.0*np.einsum('mnEf,EAfIJn->AmIJ', H.ab.oovv[:, ob, Va, vb], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('mNEf,EAfIJN->AmIJ', H.ab.oovv[:, Ob, Va, vb], T.aab.VVvOOO, optimize=True)
        +1.0*np.einsum('mneF,AeFIJn->AmIJ', H.ab.oovv[:, ob, va, Vb], T.aab.VvVOOo, optimize=True)
        +1.0*np.einsum('mNeF,AeFIJN->AmIJ', H.ab.oovv[:, Ob, va, Vb], T.aab.VvVOOO, optimize=True)
        -1.0*np.einsum('mnEF,EAFIJn->AmIJ', H.ab.oovv[:, ob, Va, Vb], T.aab.VVVOOo, optimize=True)
        -1.0*np.einsum('mNEF,EAFIJN->AmIJ', H.ab.oovv[:, Ob, Va, Vb], T.aab.VVVOOO, optimize=True)
)
I2A_VoOO += H.aa.vooo[Va, :, Oa, Oa]
get_error(I2A_vooo_full[Va, :, Oa, Oa], I2A_VoOO)

5.591020876384662e-17


In [146]:
I2A_VooO = (1.0/1.0) * (
        +0.5*np.einsum('mnef,AfeinJ->AmiJ', H.aa.oovv[:, oa, va, va], T.aaa.VvvooO, optimize=True)
        -0.5*np.einsum('mNef,AfeiJN->AmiJ', H.aa.oovv[:, Oa, va, va], T.aaa.VvvoOO, optimize=True)
        +1.0*np.einsum('mnEf,EAfinJ->AmiJ', H.aa.oovv[:, oa, Va, va], T.aaa.VVvooO, optimize=True)
        -1.0*np.einsum('mNEf,EAfiJN->AmiJ', H.aa.oovv[:, Oa, Va, va], T.aaa.VVvoOO, optimize=True)
        +0.5*np.einsum('mnEF,FEAinJ->AmiJ', H.aa.oovv[:, oa, Va, Va], T.aaa.VVVooO, optimize=True)
        -0.5*np.einsum('mNEF,FEAiJN->AmiJ', H.aa.oovv[:, Oa, Va, Va], T.aaa.VVVoOO, optimize=True)
)
I2A_VooO += (1.0/1.0) * (
        +1.0*np.einsum('mnef,AefiJn->AmiJ', H.ab.oovv[:, ob, va, vb], T.aab.VvvoOo, optimize=True)
        +1.0*np.einsum('mNef,AefiJN->AmiJ', H.ab.oovv[:, Ob, va, vb], T.aab.VvvoOO, optimize=True)
        +1.0*np.einsum('mneF,AeFiJn->AmiJ', H.ab.oovv[:, ob, va, Vb], T.aab.VvVoOo, optimize=True)
        +1.0*np.einsum('mNeF,AeFiJN->AmiJ', H.ab.oovv[:, Ob, va, Vb], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('mnEf,EAfiJn->AmiJ', H.ab.oovv[:, ob, Va, vb], T.aab.VVvoOo, optimize=True)
        -1.0*np.einsum('mNEf,EAfiJN->AmiJ', H.ab.oovv[:, Ob, Va, vb], T.aab.VVvoOO, optimize=True)
        -1.0*np.einsum('mnEF,EAFiJn->AmiJ', H.ab.oovv[:, ob, Va, Vb], T.aab.VVVoOo, optimize=True)
        -1.0*np.einsum('mNEF,EAFiJN->AmiJ', H.ab.oovv[:, Ob, Va, Vb], T.aab.VVVoOO, optimize=True)
)
I2A_VooO += H.aa.vooo[Va, :, oa, Oa]
get_error(I2A_vooo_full[Va, :, oa, Oa], I2A_VooO)

3.478482639250292e-17


In [147]:
I2A_Vooo = (1.0/1.0) * (
        -0.5*np.einsum('mNef,AfeijN->Amij', H.aa.oovv[:, Oa, va, va], T.aaa.VvvooO, optimize=True)
        -1.0*np.einsum('mNEf,EAfijN->Amij', H.aa.oovv[:, Oa, Va, va], T.aaa.VVvooO, optimize=True)
        -0.5*np.einsum('mNEF,FEAijN->Amij', H.aa.oovv[:, Oa, Va, Va], T.aaa.VVVooO, optimize=True)
)
I2A_Vooo += (1.0/1.0) * (
        +1.0*np.einsum('mNef,AefijN->Amij', H.ab.oovv[:, Ob, va, vb], T.aab.VvvooO, optimize=True)
        +1.0*np.einsum('mNeF,AeFijN->Amij', H.ab.oovv[:, Ob, va, Vb], T.aab.VvVooO, optimize=True)
        -1.0*np.einsum('mNEf,EAfijN->Amij', H.ab.oovv[:, Ob, Va, vb], T.aab.VVvooO, optimize=True)
        -1.0*np.einsum('mNEF,EAFijN->Amij', H.ab.oovv[:, Ob, Va, Vb], T.aab.VVVooO, optimize=True)
)
I2A_Vooo += H.aa.vooo[Va, :, oa, oa]
get_error(I2A_vooo[Va, :, oa, oa], I2A_Vooo)

3.444567207626637e-17


In [148]:
I2A_voOO = (1.0/1.0) * (
        +1.0*np.einsum('mnEf,EfanIJ->amIJ', H.aa.oovv[:, oa, Va, va], T.aaa.VvvoOO, optimize=True)
        -0.5*np.einsum('mnEF,FEanIJ->amIJ', H.aa.oovv[:, oa, Va, Va], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('mNEf,EfaIJN->amIJ', H.aa.oovv[:, Oa, Va, va], T.aaa.VvvOOO, optimize=True)
        -0.5*np.einsum('mNEF,FEaIJN->amIJ', H.aa.oovv[:, Oa, Va, Va], T.aaa.VVvOOO, optimize=True)
)
I2A_voOO += (1.0/1.0) * (
        -1.0*np.einsum('mneF,eaFIJn->amIJ', H.ab.oovv[:, ob, va, Vb], T.aab.vvVOOo, optimize=True)
        -1.0*np.einsum('mnEf,EafIJn->amIJ', H.ab.oovv[:, ob, Va, vb], T.aab.VvvOOo, optimize=True)
        -1.0*np.einsum('mnEF,EaFIJn->amIJ', H.ab.oovv[:, ob, Va, Vb], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('mNeF,eaFIJN->amIJ', H.ab.oovv[:, Ob, va, Vb], T.aab.vvVOOO, optimize=True)
        -1.0*np.einsum('mNEf,EafIJN->amIJ', H.ab.oovv[:, Ob, Va, vb], T.aab.VvvOOO, optimize=True)
        -1.0*np.einsum('mNEF,EaFIJN->amIJ', H.ab.oovv[:, Ob, Va, Vb], T.aab.VvVOOO, optimize=True)
)
I2A_voOO += H.aa.vooo[va, :, Oa, Oa]
get_error(I2A_vooo_full[va, :, Oa, Oa], I2A_voOO)

2.254777840137041e-18


In [149]:
I2A_voOo = (1.0/1.0) * (
        -1.0*np.einsum('mneF,FeajnI->amIj', H.aa.oovv[:, oa, va, Va], T.aaa.VvvooO, optimize=True)
        -0.5*np.einsum('mnEF,FEajnI->amIj', H.aa.oovv[:, oa, Va, Va], T.aaa.VVvooO, optimize=True)
        +1.0*np.einsum('mNeF,FeajIN->amIj', H.aa.oovv[:, Oa, va, Va], T.aaa.VvvoOO, optimize=True)
        +0.5*np.einsum('mNEF,FEajIN->amIj', H.aa.oovv[:, Oa, Va, Va], T.aaa.VVvoOO, optimize=True)
)
I2A_voOo += (1.0/1.0) * (
        +1.0*np.einsum('mnEf,EafjIn->amIj', H.ab.oovv[:, ob, Va, vb], T.aab.VvvoOo, optimize=True)
        +1.0*np.einsum('mneF,eaFjIn->amIj', H.ab.oovv[:, ob, va, Vb], T.aab.vvVoOo, optimize=True)
        +1.0*np.einsum('mnEF,EaFjIn->amIj', H.ab.oovv[:, ob, Va, Vb], T.aab.VvVoOo, optimize=True)
        +1.0*np.einsum('mNEf,EafjIN->amIj', H.ab.oovv[:, Ob, Va, vb], T.aab.VvvoOO, optimize=True)
        +1.0*np.einsum('mNeF,eaFjIN->amIj', H.ab.oovv[:, Ob, va, Vb], T.aab.vvVoOO, optimize=True)
        +1.0*np.einsum('mNEF,EaFjIN->amIj', H.ab.oovv[:, Ob, Va, Vb], T.aab.VvVoOO, optimize=True)
)
I2A_voOo += H.aa.vooo[va, :, Oa, oa]
get_error(I2A_vooo_full[va, :, Oa, oa], I2A_voOo)

2.2814809664410264e-17


In [150]:
I2A_vooo = (1.0/1.0) * (
        +1.0*np.einsum('mNEf,EfaijN->amij', H.aa.oovv[:, Oa, Va, va], T.aaa.VvvooO, optimize=True)
        -0.5*np.einsum('mNEF,FEaijN->amij', H.aa.oovv[:, Oa, Va, Va], T.aaa.VVvooO, optimize=True)
)
I2A_vooo += (1.0/1.0) * (
        -1.0*np.einsum('mNeF,eaFijN->amij', H.ab.oovv[:, Ob, va, Vb], T.aab.vvVooO, optimize=True)
        -1.0*np.einsum('mNEf,EafijN->amij', H.ab.oovv[:, Ob, Va, vb], T.aab.VvvooO, optimize=True)
        -1.0*np.einsum('mNEF,EaFijN->amij', H.ab.oovv[:, Ob, Va, Vb], T.aab.VvVooO, optimize=True)
)
I2A_vooo += H.aa.vooo[va, :, oa, oa]
get_error(I2A_vooo_full[va, :, oa, oa], I2A_vooo)


2.197696792124875e-17


In [153]:
# assemble full intermediate
I2A_vooo_2 = np.zeros(shape=I2A_vooo_full.shape)

I2A_vooo_2[Va, :, Oa, Oa] = I2A_VoOO
I2A_vooo_2[Va, :, oa, Oa] = I2A_VooO; I2A_vooo_2[Va, :, Oa, oa] = -1.0*np.transpose(I2A_VooO, (0, 1, 3, 2));
I2A_vooo_2[Va, :, oa, oa] = I2A_Vooo
I2A_vooo_2[va, :, Oa, Oa] = I2A_voOO
I2A_vooo_2[va, :, Oa, oa] = I2A_voOo; I2A_vooo_2[va, :, oa, Oa] = -1.0*np.transpose(I2A_voOo, (0, 1, 3, 2));
I2A_vooo_2[va, :, oa, oa] = I2A_vooo

In [154]:
get_error(I2A_vooo_2, I2A_vooo_full)

9.089293858866171e-17
